# =============
# 数据采集自检程序 1.7.0
## 一.基础信息检查
## 二.title、raw_title字段检查
## 三. keywords、terms字段查询
## 四. authors字段检查
## 五. src、sid、lang、venue、ts字段检查
## 六. issn、isbn、doi、pdf_src、paper_type、editor字段检查
## 七. year、volume、issue、page_start、page_end、page_str字段查询
# =============

In [1]:
import numpy as np
import pandas as pd
import json
import re
import time
from collections import Counter

np.set_printoptions(threshold=np.inf)
pd.set_option('display.width',None) 

In [2]:
data = pd.read_json('F:/journals/JOURNAL_OF_AMERICAN_COLLEGE_HEALTH.json',encoding='utf-8',lines=False)

In [3]:
data.head()

,_id,title,raw_title,year,volume,src,page_str,page_start,page_end,issn,...,ts,abstract,lang,authors,issue,paper_type,reference,citation,keywords,notes
0,{'$oid': '5e1705c8313804539c62ab81'},Receipt of the Human Papillomavirus Vaccine Am...,"<span class=""NLM_article-title hlFld-title"">Re...",2013,61,Taylor & Francis Group,18-27,18,27,0744-8481,...,{'$date': '2020-01-09T18:51:52.000+0000'},"<div class=""abstractSection abstractInFull""> <...",en,"[{'pos': 0, 'name': 'Lisa L. Lindley', 'org': ...",1,research-article,"[<li id=""CIT0001""><span>Centers for Disease Co...","[<div class=""citedByEntry""><span class=""entryA...",clinical medicine;gender;health education,NaN
1,{'$oid': '5e1705c8313804539c62ab82'},Quantifying the magnitude and potential influe...,"<span class=""NLM_article-title hlFld-title"">Qu...",2019,67,Taylor & Francis,42-50,42,50,0744-8481,...,{'$date': '2020-01-09T18:51:52.000+0000'},"<div class=""abstractSection abstractInFull""><p...",en,"[{'pos': 0, 'name': 'Molly Rosenberg', 'org': ...",1,research-article,"[<li id=""cit0001""><span><span class=""hlFld-Con...","[<div class=""citedByEntry""><span class=""entryA...",Campus sexual assault;gender;missing data;resp...,NaN
2,{'$oid': '5e1705c8313804539c62ab83'},Perspectives on College Health in the 80's: A ...,"<span class=""NLM_article-title hlFld-title"">Pe...",1981,30,Taylor & Francis Group,11-13,11,13,0744-8481,...,{'$date': '2020-01-09T18:51:52.000+0000'},NaN,en,"[{'pos': 0, 'name': 'Harold L. Enarson', 'org'...",1,research-article,NaN,NaN,NaN,NaN
3,{'$oid': '5e1705c8313804539c62ab84'},Drunkorexia: A new term and diagnostic criteria,"<span class=""NLM_article-title hlFld-title"">Dr...",2019,67,Taylor & Francis,620-626,620,626,0744-8481,...,{'$date': '2020-01-09T18:51:52.000+0000'},"<div class=""abstractSection abstractInFull""><p...",en,"[{'pos': 0, 'name': 'Carly Thompson-Memmer', '...",7,opinion,"[<li id=""CIT0001""><span><span class=""hlFld-Con...","[<div class=""citedByEntry""><span class=""entryA...",College students;disordered eating;drunkorexia...,NaN
4,{'$oid': '5e1705c8313804539c62ab85'},Comparative Study of the Opinions and Beliefs ...,"<span class=""NLM_article-title hlFld-title"">Co...",1981,29,Taylor & Francis Group,233-235,233,235,0744-8481,...,{'$date': '2020-01-09T18:51:52.000+0000'},"<div class=""abstractSection abstractInFull""> <...",en,"[{'pos': 0, 'name': 'Ikponmwosa Owie', 'org': ...",5,research-article,"[<li id=""CIT0001""><span><span class=""hlFld-Con...",NaN,NaN,NaN


# =============
# 一. 基础信息查询
# =============

## 1. 数据总数

In [4]:
print('数据总量：',len(data))

数据总量： 2484


## 2. 数据包含字段

In [5]:
print('数据包含的字段名称如下')
data_str_len = len(data.columns)
for i in range(data_str_len):    
    print(data.columns[i])
print('数据字段数目：' + str(data_str_len) + '\n')

importion_str = ['url','hash','title','raw_title','year','lang','src','sid','ts']
importion_str_tag = 0
for i in range(len(importion_str)):
    if importion_str[i] not in data.columns:
        print('--------------------')
        print('异常，可能缺少必填字段' + importion_str[i])
    else:
        importion_str_tag = 1
if importion_str_tag == 0:
    print('正常，不缺少任何必填字段！')

数据包含的字段名称如下
_id
title
raw_title
year
volume
src
page_str
page_start
page_end
issn
doi
sid
url
pdf_src
venue
hash
ts
abstract
lang
authors
issue
paper_type
reference
citation
keywords
notes
数据字段数目：26



## 3. 检查是否存在重复数据

In [6]:
urllist = []
for url in data['url']:
    urllist.append(url[0])

url_list = list(set(urllist))

if len(url_list) == len(urllist):
    print('正常，未发现重复数据！')
else:
    print('异常，可能存在重复数据')
    print('【重复数据量 / 数据总量】  <==>  【',len(urllist)-len(url_list),'/',len(data),'】\n')

    #展现重复元素和重复次数
    repeation_tag = dict(Counter(urllist))
    for key,value in repeation_tag.items():
        if value>1:
            print('url：',key,'\t出现次数：',value)

正常，未发现重复数据！


## 4. 检查url的数据类型（url-List[String]）

In [7]:
if 'url' in data.columns:
    have_url = data[data['url'].notnull()]
    url_type = []
    url_type_warn = 0
    for i in range(len(have_url)):
        url_type.append(str(isinstance(have_url.iloc[i]['url'],list)))
        if not isinstance(have_url.iloc[i]['url'],list):
            url_type_warn = url_type_warn + 1
    if 'False' not in url_type:
        print('正常，url数据类型正确')
    else:
        print('异常，url数据类型可能存在错误')
        print('【url数据类型可能存在错误的数据量 / 存在url的数据总量】  <==>  【',url_type_warn,'/',len(have_url),'】\n')
        for i in range(len(have_url)):
            if not isinstance(have_url.iloc[i]['url'],list):
                print('出现问题的url数据类型：',type(have_url.iloc[i]['url']))
else:
    print('异常，可能缺少必填字段url')

正常，url数据类型正确


# =============
# 二. title, raw_title字段查询
# =============

## 1. 检查title和raw_title的数据类型（title-String、raw_title-Html String）

**(1) 检查 title的数据类型**

In [8]:
if 'title' in data.columns:
    have_title = data[data['title'].notnull()]
    title_type = []
    title_type_warn = 0
    for i in range(len(have_title)):
        title_type.append(str(isinstance(have_title.iloc[i]['title'],str)))
        if not isinstance(have_title.iloc[i]['title'],str):
            title_type_warn = title_type_warn + 1
    if 'False' not in title_type:
        print('正常，title数据类型正确')
    else:
        print('异常，title数据类型可能存在错误')
        print('【title数据类型可能存在错误的数据量 / 存在title的数据总量】  <==>  【',title_type_warn,'/',len(have_title),'】\n')
        for i in range(len(have_title)):
            if not isinstance(have_title.iloc[i]['title'],str):
                print('出现问题的title数据类型：',type(have_title.iloc[i]['title']))
                print('url：',have_title.iloc[i]['url'][0])
else:
    print('异常，可能缺少必填字段title')

正常，title数据类型正确


**(2) 检查 raw_title的数据类型**

In [9]:
if 'raw_title' in data.columns:
    have_raw_title = data[data['raw_title'].notnull()]
    raw_title_type = []
    raw_title_type_warn = 0

    for i in range(len(have_raw_title)):
        raw_title_type.append(str(isinstance(have_raw_title.iloc[i]['raw_title'],str)))
        if not isinstance(have_raw_title.iloc[i]['raw_title'],str):
            raw_title_type_warn = raw_title_type_warn + 1
        else:
            if (have_raw_title.iloc[i]['raw_title'][0] != '<') or (have_raw_title.iloc[i]['raw_title'][-1] != '>') :
                raw_title_type_warn = raw_title_type_warn + 1
    if 'False' not in raw_title_type:
        print('正常，raw_title数据类型正确')
    else:
        print('异常，raw_title数据类型可能存在错误')
        print('【raw_title数据类型可能存在错误的数据量 / 存在raw_title的数据总量】  <==>  【',raw_title_type_warn,'/',len(have_raw_title),'】\n')
        for i in range(len(have_raw_title)):
            if not isinstance(have_raw_title.iloc[i]['raw_title'],str):
                print('出现问题的raw_title数据类型：',type(have_raw_title.iloc[i]['raw_title']))
                print('url：',have_raw_title.iloc[i]['url'][0],'\n')
            else:
                if (have_raw_title.iloc[i]['raw_title'][0] != '<') or (have_raw_title.iloc[i]['raw_title'][-1] != '>') :
                    print('raw_title可能非Html String类型：')
                    print('url：',have_raw_title.iloc[i]['url'][0],'\n')
else:
    print('异常，可能缺少必填字段raw_title')

正常，raw_title数据类型正确


## 2. 检查raw_title和title的数量、缺失情况

**(1) 检查 没有title字段，没有raw_title字段 情况**

In [10]:
if ('title' in data.columns) and ('raw_title' in data.columns):
    no_title = data[data['title'].isnull()]
    no_title_no_raw_title = no_title[no_title['raw_title'].isnull()]

    if len(no_title_no_raw_title):
        print('异常，可能存在没有title，没有raw_title的情况')
        print('【没有title，没有raw_title的数据量 / 数据总量】  <==>  【',len(no_title_no_raw_title),'/',len(data),'】\n')
        print('请详细检查以下数据源：')
        for i in range(len(no_title_no_raw_title)):
            print('url：',no_title_no_raw_title.iloc[i]['url'][0])
    else:
        print('正常，不存在没有title，没有raw_title的情况')
else:
    if 'title' not in data.columns:
        print('异常，可能缺少必填字段title')
    if 'raw_title' not in data.columns:
        print('异常，可能缺少必填字段raw_title')

正常，不存在没有title，没有raw_title的情况


**(2) 检查 有title字段，没有raw_title字段 情况**

In [11]:
if ('title' in data.columns) and ('raw_title' in data.columns):
    have_title = data[data['title'].notnull()]
    have_title_no_raw_title = have_title[have_title['raw_title'].isnull()]

    if len(have_title_no_raw_title):
        print('异常，可能存在有title，没有raw_title的情况\n')
        print('【有title，没有raw_title的数据量 / 数据总量】  <==>  【',len(have_title_no_raw_title),'/',len(data),'】\n')
        print('请详细检查以下数据源：')
        for i in range(len(have_title_no_raw_title)):
            print('url：',have_title_no_raw_title.iloc[i]['url'][0])
    else:
        print('正常，不存在有title，没有raw_title的情况')
else:
    if 'title' not in data.columns:
        print('异常，可能缺少必填字段title')
    if 'raw_title' not in data.columns:
        print('异常，可能缺少必填字段raw_title')

正常，不存在有title，没有raw_title的情况


**(3) 检查 没有title字段，有raw_title字段 情况**

In [12]:
if ('title' in data.columns) and ('raw_title' in data.columns):
    have_raw_title = data[data['raw_title'].notnull()]
    have_raw_title_no_title = have_raw_title[have_raw_title['title'].isnull()]

    if len(have_raw_title_no_title):
        print('异常，可能存在没有title，有raw_title的情况\n')
        print('【没有title，有raw_title的数据量 / 数据总量】  <==>  【',len(have_raw_title_no_title),'/',len(data),'】\n')
        print('请详细检查以下数据源：')
        for i in range(len(have_raw_title_no_title)):
            print('url：',have_raw_title_no_title.iloc[i]['url'][0])
    else:
        print('正常，不存在没有title，有raw_title的情况')
else:
    if 'title' not in data.columns:
        print('异常，可能缺少必填字段title')
    if 'raw_title' not in data.columns:
        print('异常，可能缺少必填字段raw_title')

正常，不存在没有title，有raw_title的情况


## 3. 检查raw_title和title常见问题

**(1) 检查 title字段为空情况**

In [13]:
if 'title' in data.columns:
    have_title = data[data['title'].notnull()]
    tag = 0
    for i in range(len(have_title)):
        title = have_title.iloc[i]['title'].strip()
        if title == '':
            if tag == 0:
                tag = 1
                print('异常，可能存在title字段为空的情况\n')
                print('请详细检查以下数据源：')
            print('url：',have_title.iloc[i]['url'][0])
    if tag == 0:
        print('正常，未发现title字段为空的情况')
else:
    print('异常，可能缺少必填字段title')

正常，未发现title字段为空的情况


**(2) 检查 raw_title字段为空情况**

In [14]:
if 'raw_title' in data.columns:
    have_raw_title = data[data['raw_title'].notnull()]
    tag = 0
    for i in range(len(have_raw_title)):
        raw_title = have_raw_title.iloc[i]['raw_title'].strip()
        if raw_title == '':
            if tag == 0:
                tag = 1
                print('异常，可能存在raw_title字段为空的情况\n')
                print('请详细检查以下数据源：')
            print('url：', have_raw_title.iloc[i]['url'][0])
    if tag == 0:
        print('正常，未发现raw_title字段为空的情况')
else:
    print('异常，可能缺少必填字段raw_title')

正常，未发现raw_title字段为空的情况


**(3) 检查title字段内容是否存在常见问题**

In [15]:
if 'title' in data.columns:
    have_title = data[data['title'].notnull()]
    tag = 0
    for i in range(len(have_title)):
        title = have_title.iloc[i]['title']
        if ('\n' in title) or ('\t' in title) or ('\r' in title) or ('  ' in title) or (not title[0].isalnum()) or (title[-1]==' '):
            tag = tag + 1
            if tag == 1:
                print('异常，title字段中可能存在问题')
            if ('\n' in title) or ('\t' in title) or ('\r' in title):
                print('title：',have_title.iloc[i]['title'])
                print('title可能存在转义字符')
                print('url：',have_title.iloc[i]['url'][0],'\n')
            if ('  ' in title) or (title[0] == ' ') or (title[-1] == ' '):
                print('title：',have_title.iloc[i]['title'])
                print('title可能存在多余空格')
                print('url：',have_title.iloc[i]['url'][0],'\n')
            if not title[0].isalnum():
                if title[0] != ' ':
                    print('title：',have_title.iloc[i]['title'])
                    print('title中存在异常')
                    print('url：',have_title.iloc[i]['url'][0],'\n')
    if tag == 0:
        print('正常，title字段未发现常见问题')
else:
    print('异常，可能缺少必填字段title')

异常，title字段中可能存在问题
title： "Drinking is Fun" and "There's Nothing You Can Do About It": The Problem With the 21-Year-Old Minimum Drinking Age
title中存在异常
url： https://www.tandfonline.com/doi/abs/10.3200/JACH.56.3.213-216 

title： “Alcohol-Related Vehicular Death Rates for College Students in the Commonwealth of Virginia”
title中存在异常
url： https://www.tandfonline.com/doi/full/10.1080/07448481.2011.576294 

title： “Consent is Good, Joyous, Sexy”: A banner campaign to market consent to college students
title中存在异常
url： https://www.tandfonline.com/doi/full/10.1080/07448481.2016.1217869 

title： “This is not a drill”: Activation of a student-led influenza vaccination point of dispensing
title中存在异常
url： https://www.tandfonline.com/doi/full/10.1080/07448481.2018.1463228 

title： “Consent is sexy”: A poster campaign using sex-positive images and messages to increase dyadic sexual communication
title中存在异常
url： https://www.tandfonline.com/doi/full/10.1080/07448481.2018.1515746 

title： “Drunkorexia”: 

## 4. 浏览检查title和raw_title

**(1) 浏览检查 title字段**

In [16]:
if 'title' in data.columns:
    have_title = data[data['title'].notnull()]
    print('请浏览检查title字段：\n')
    for i in range(len(have_title)):
        print('title：',have_title.iloc[i]['title'])
        print('url：',have_title.iloc[i]['url'][0],'\n')
else:
    print('异常，可能缺少必填字段title')

请浏览检查title字段：

title： Receipt of the Human Papillomavirus Vaccine Among Female College Students in the United States, 2009
url： https://www.tandfonline.com/doi/full/10.1080/07448481.2012.750607 

title： Quantifying the magnitude and potential influence of missing data in campus sexual assault surveys: A systematic review of surveys, 2010–2016
url： https://www.tandfonline.com/doi/full/10.1080/07448481.2018.1462817 

title： Perspectives on College Health in the 80's: A President's View
url： https://www.tandfonline.com/doi/abs/10.1080/01644300.1981.10393046 

title： Drunkorexia: A new term and diagnostic criteria
url： https://www.tandfonline.com/doi/full/10.1080/07448481.2018.1500470 

title： Comparative Study of the Opinions and Beliefs of Undergraduate Students in Nigeria and the United States toward Family Size
url： https://www.tandfonline.com/doi/abs/10.1080/01644300.1981.10393017 

title： Nonmedical Prescription Drug Use Among College Students: A Comparison Between Athletes and Nonat

title： The Political, Social, and Public Health Problems of Binge Drinking in College
url： https://www.tandfonline.com/doi/abs/10.1080/07448480009599304 

title： Epidemiologic Aspects of College Mental Health
url： https://www.tandfonline.com/doi/abs/10.3200/JACH.54.6.372-376 

title： Relationships between use of dietary supplements, caffeine and sensation seeking among college students
url： https://www.tandfonline.com/doi/full/10.1080/07448481.2018.1500475 

title： Gaining Perspective
url： https://www.tandfonline.com/doi/abs/10.1080/07448481.1995.9940913 

title： Guest Reviewers
url： https://www.tandfonline.com/doi/abs/10.1080/07448481.1985.9935032 

title： The Health of University Athletes: Attitudes, Behaviors, and Stressors
url： https://www.tandfonline.com/doi/abs/10.1080/07448481.1990.9936208 

title： Changing with the Times
url： https://www.tandfonline.com/doi/abs/10.1080/07448481.1989.9939068 

title： Research in College Health I: An Introduction to the Research Process
url： http

title： Commentary on Antiandrogenic Treatment in a University Setting
url： https://www.tandfonline.com/doi/abs/10.1080/07448481.1987.9939644 

title： Peer-Based HIV Risk Assessment: A Step-By-Step Guide Through the Teachable Moment
url： https://www.tandfonline.com/doi/abs/10.1080/07448481.1993.9936352 

title： Exposing College Students to Exercise: The Training Interventions and Genetics of Exercise Response (TIGER) Study
url： https://www.tandfonline.com/doi/full/10.1080/07448481.2010.483712 

title： Managing Helicobacter pylori in college health, with special considerations for international students
url： https://www.tandfonline.com/doi/full/10.1080/07448481.2015.1085053 

title： Understanding Physical Activity Behavior in African American and Caucasian College Students: An Application of the Theory of Planned Behavior
url： https://www.tandfonline.com/doi/abs/10.3200/JACH.56.44.341-346 

title： Reviewer Acknowledgment
url： https://www.tandfonline.com/doi/full/10.1080/07448481.2015.101

url： https://www.tandfonline.com/doi/abs/10.1080/07448489809595637 

title： College Student Behavior: A Ten-Year Look
url： https://www.tandfonline.com/doi/abs/10.1080/07448481.1983.9939559 

title： Annual Index: Volume 56 July/August 2007-May/June 2008
url： https://www.tandfonline.com/doi/abs/10.3200/JACH.56.6.716-724 

title： Identifying Male College Students' Perceived Health Needs, Barriers to Seeking Help, and Recommendations to Help Men Adopt Healthier Lifestyles
url： https://www.tandfonline.com/doi/abs/10.1080/07448480009596267 

title： Characteristics of College Students Who Use Psychoactive Drugs for Nonmedical Reasons
url： https://www.tandfonline.com/doi/abs/10.1080/07448481.1985.9939603 

title： Depressive Symptoms and Mental Health Treatment in an Ethnoracially Diverse College Student Sample
url： https://www.tandfonline.com/doi/full/10.1080/07448481.2010.529625 

title： An Introduction to the Lexington Report on Blood-Borne Pathogen Disease in Health Science Students
url： ht

url： https://www.tandfonline.com/doi/full/10.1080/07448481.2013.872648 

title： Living Arrangements, Knowledge of Health Risks, and Stress as Determinants of Health-Risk Behavior Among College Students
url： https://www.tandfonline.com/doi/abs/10.1080/07448481.1992.10392817 

title： Understanding Relationships: The Next Critical Challenge
url： https://www.tandfonline.com/doi/abs/10.1080/07448489809600222 

title： ACHA Guidelines
url： https://www.tandfonline.com/doi/full/10.1080/07448481.2016.1141017 

title： Campus sexual assault climate surveys: A brief exploration of publicly available reports
url： https://www.tandfonline.com/doi/full/10.1080/07448481.2018.1431914 

title： Characteristics of university students who mix alcohol and energy drinks
url： https://www.tandfonline.com/doi/full/10.1080/07448481.2017.1280799 

title： Commentary
url： https://www.tandfonline.com/doi/abs/10.1080/07448481.1987.9939004 

title： Gonococcal Perihepatitis: Diagnosis and Management in a College Populati

url： https://www.tandfonline.com/doi/abs/10.1080/07448481.1989.9938423 

title： A View Through a Different Lens: Photovoice as a Tool for Student Advocacy
url： https://www.tandfonline.com/doi/abs/10.3200/JACH.55.1.53-56 

title： Problem Areas in the Management of Urinary Tract Infections
url： https://www.tandfonline.com/doi/abs/10.1080/07448481.1987.9939010 

title： Keynote Address
url： https://www.tandfonline.com/doi/abs/10.1080/01644300.1979.10392889 

title： Should College Campuses Become Tobacco Free Without an Enforcement Plan?
url： https://www.tandfonline.com/doi/full/10.1080/07448481.2012.716981 

title： Brief Screening and Intervention for Alcohol and Drug Use in a College Student Health Clinic: Feasibility, Implementation, and Outcomes
url： https://www.tandfonline.com/doi/full/10.1080/07448480903501764 

title： Development and Validation of a Condom Self-Efficacy Scale for College Students
url： https://www.tandfonline.com/doi/abs/10.1080/07448481.1991.9936238 

title： College 

url： https://www.tandfonline.com/doi/full/10.1080/07448481.2015.1015030 

title： The Evolution of an Eye Care Program at a University Health Service
url： https://www.tandfonline.com/doi/abs/10.1080/07448481.1981.9938880 

title： Dental Attitudes, Perceptions, and Treatment Needs in a University Population
url： https://www.tandfonline.com/doi/abs/10.1080/01644300.1980.10392977 

title： General Statement of Ethical Principles and Guidelines
url： https://www.tandfonline.com/doi/full/10.1080/07448481.2011.497050 

title： Genital Chlamydial Infection in Female and Male College Students
url： https://www.tandfonline.com/doi/abs/10.1080/07448481.1989.9937497 

title： Editorial Board Page
url： https://www.tandfonline.com/doi/abs/10.1080/01644300.1981.10393049 

title： Immunization documentation practices and vaccine-preventable disease surveillance capacity among institutions of higher education in Indiana
url： https://www.tandfonline.com/doi/full/10.1080/07448481.2017.1381967 

title： Mental H

url： https://www.tandfonline.com/doi/abs/10.1080/07448481.1984.9936170 

title： Phencyclidine Hydrochloride (PCP) Use Among College Students: Subjective and Clinical Effects, Toxicity, Diagnosis, and Treatment
url： https://www.tandfonline.com/doi/abs/10.1080/07448481.1984.10392806 

title： The Implementation of Buprenorphine/Naloxone in College Health Practice
url： https://www.tandfonline.com/doi/abs/10.3200/JACH.56.44.391-394 

title： Pharyngitis in the University Population
url： https://www.tandfonline.com/doi/abs/10.1080/07448481.1984.10392811 

title： Editorial Board Page
url： https://www.tandfonline.com/doi/abs/10.1080/01644300.1979.10392893 

title： Contraceptive Responsibility among Male University Students
url： https://www.tandfonline.com/doi/abs/10.1080/01644300.1979.10392925 

title： Elimination of After-Hours Care at a University Health Service
url： https://www.tandfonline.com/doi/abs/10.1080/07448481.1990.9936220 

title： Effects of Gender and Motivations on Perceptions of 

url： https://www.tandfonline.com/doi/full/10.1080/07448481.2013.818000 

title： Development, Evaluation, and Validation of Environmental Assessment Tools to Evaluate the College Nutrition Environment
url： https://www.tandfonline.com/doi/full/10.1080/07448481003686026 

title： Managing chronic conditions in college: Findings from prompted health incidents diaries
url： https://www.tandfonline.com/doi/full/10.1080/07448481.2016.1266640 

title： Abstracts of the Scientific Programs of the American College Health Association Fifty-Ninth Annual Meeting April 21–24, 1981, Boston, Massachusetts
url： https://www.tandfonline.com/doi/abs/10.1080/01644300.1981.10393047 

title： Relationships Between Physical Activity and the Proximity of Exercise Facilities and Home Exercise Equipment Used by Undergraduate University Students
url： https://www.tandfonline.com/doi/abs/10.3200/JACH.53.6.285-290 

title： Moving Targets: Bicycle-Related Injuries and Helmet Use among University Students
url： https://www

**(2) 浏览检查 raw_title字段**

In [17]:
if "raw_title" in data.columns:
    have_raw_title = data[data['raw_title'].notnull()]
    print('请浏览检查raw_title字段：\n')
    for i in range(len(have_raw_title)):
        print('raw_title：',have_raw_title.iloc[i]['raw_title'])
        print('url：',have_raw_title.iloc[i]['url'][0],'\n')
else:
    print('异常，可能缺少必填字段raw_title')

请浏览检查raw_title字段：

raw_title： <span class="NLM_article-title hlFld-title">Receipt of the Human Papillomavirus Vaccine Among Female College Students in the United States, 2009</span>
url： https://www.tandfonline.com/doi/full/10.1080/07448481.2012.750607 

raw_title： <span class="NLM_article-title hlFld-title">Quantifying the magnitude and potential influence of missing data in campus sexual assault surveys: A systematic review of surveys, 2010–2016</span>
url： https://www.tandfonline.com/doi/full/10.1080/07448481.2018.1462817 

raw_title： <span class="NLM_article-title hlFld-title">Perspectives on College Health in the 80's: A President's View</span>
url： https://www.tandfonline.com/doi/abs/10.1080/01644300.1981.10393046 

raw_title： <span class="NLM_article-title hlFld-title">Drunkorexia: A new term and diagnostic criteria</span>
url： https://www.tandfonline.com/doi/full/10.1080/07448481.2018.1500470 

raw_title： <span class="NLM_article-title hlFld-title">Comparative Study of the Opin

raw_title： <span class="NLM_article-title hlFld-title">Adolescent Risk Behaviors and the Potential for Violence: A Look at What's Coming to Campus</span>
url： https://www.tandfonline.com/doi/abs/10.1080/07448481.1993.9936315 

raw_title： <span class="NLM_article-title hlFld-title">The Political, Social, and Public Health Problems of Binge Drinking in College</span>
url： https://www.tandfonline.com/doi/abs/10.1080/07448480009599304 

raw_title： <span class="NLM_article-title hlFld-title">Epidemiologic Aspects of College Mental Health</span>
url： https://www.tandfonline.com/doi/abs/10.3200/JACH.54.6.372-376 

raw_title： <span class="NLM_article-title hlFld-title">Relationships between use of dietary supplements, caffeine and sensation seeking among college students</span>
url： https://www.tandfonline.com/doi/full/10.1080/07448481.2018.1500475 

raw_title： <span class="NLM_article-title hlFld-title">Gaining Perspective</span>
url： https://www.tandfonline.com/doi/abs/10.1080/07448481.1995.

raw_title： <span class="NLM_article-title hlFld-title">The State of the Union: Sexual Health Disparities in a National Sample of US College Students</span>
url： https://www.tandfonline.com/doi/full/10.1080/07448480903501780 

raw_title： <span class="NLM_article-title hlFld-title">Providing Student Health Services at Sea: A Survey of Chief Complaints Onboard a Maritime Academy Training Ship</span>
url： https://www.tandfonline.com/doi/abs/10.3200/JACH.57.4.457-464 

raw_title： <span class="NLM_article-title hlFld-title">College Health Responds to HIV: Hard Lessons and a Rich Heritage</span>
url： https://www.tandfonline.com/doi/abs/10.1080/07448481.1995.9940903 

raw_title： <span class="NLM_article-title hlFld-title">Perspectives on Personal Protective Equipment</span>
url： https://www.tandfonline.com/doi/abs/10.1080/07448489909595647 

raw_title： <span class="NLM_article-title hlFld-title">Irritable Bowel Syndrome in the College Population: Pathophysiology, Differential Diagnosis, and Tr

raw_title： <span class="NLM_article-title hlFld-title">College Students' Use of Emergency Medical Services</span>
url： https://www.tandfonline.com/doi/abs/10.1080/07448481.1990.9936202 

raw_title： <span class="NLM_article-title hlFld-title">Tobacco, Depression, and Lifestyle Choices in the Pivotal Early College Years</span>
url： https://www.tandfonline.com/doi/abs/10.3200/JACH.52.5.221-228 

raw_title： <span class="NLM_article-title hlFld-title">Transgender college students: Academic resilience and striving to cope in the face of marginalized health</span>
url： https://www.tandfonline.com/doi/full/10.1080/07448481.2018.1465060 

raw_title： <span class="NLM_article-title hlFld-title">US College Students' Reports of Receiving Health Information on College Campuses</span>
url： https://www.tandfonline.com/doi/abs/10.1080/07448480109596307 

raw_title： <span class="NLM_article-title hlFld-title">Drinking Game Participation Among Undergraduate Students Attending National Alcohol Screening D


raw_title： <span class="NLM_article-title hlFld-title">Book Review</span>
url： https://www.tandfonline.com/doi/abs/10.1080/07448489809595638 

raw_title： <span class="NLM_article-title hlFld-title">Modest Reassurance in a Season of Sadness: Guns in College</span>
url： https://www.tandfonline.com/doi/abs/10.1080/07448489909595665 

raw_title： <span class="NLM_article-title hlFld-title">Effect of warning placement on the information processing of college students reading an OTC drug facts panel</span>
url： https://www.tandfonline.com/doi/full/10.1080/07448481.2018.1431891 

raw_title： <span class="NLM_article-title hlFld-title">Mental Health Need, Awareness, and Use of Counseling Services Among International Graduate Students</span>
url： https://www.tandfonline.com/doi/abs/10.3200/JACH.56.2.109-118 

raw_title： <span class="NLM_article-title hlFld-title">Cardiovascular risk factors among college students: Knowledge, perception, and risk assessment</span>
url： https://www.tandfonline.com

url： https://www.tandfonline.com/doi/abs/10.1080/07448481.1989.9938423 

raw_title： <span class="NLM_article-title hlFld-title">A View Through a Different Lens: Photovoice as a Tool for Student Advocacy</span>
url： https://www.tandfonline.com/doi/abs/10.3200/JACH.55.1.53-56 

raw_title： <span class="NLM_article-title hlFld-title">Problem Areas in the Management of Urinary Tract Infections</span>
url： https://www.tandfonline.com/doi/abs/10.1080/07448481.1987.9939010 

raw_title： <span class="NLM_article-title hlFld-title">Keynote Address</span>
url： https://www.tandfonline.com/doi/abs/10.1080/01644300.1979.10392889 

raw_title： <span class="NLM_article-title hlFld-title">Should College Campuses Become Tobacco Free Without an Enforcement Plan?</span>
url： https://www.tandfonline.com/doi/full/10.1080/07448481.2012.716981 

raw_title： <span class="NLM_article-title hlFld-title">Brief Screening and Intervention for Alcohol and Drug Use in a College Student Health Clinic: Feasibility, Implem

url： https://www.tandfonline.com/doi/abs/10.1080/07448480009596273 

raw_title： <span class="NLM_article-title hlFld-title">Incidence of Alcohol Abuse in the Parents of Medical Students</span>
url： https://www.tandfonline.com/doi/abs/10.1080/07448481.1993.9940465 

raw_title： <span class="NLM_article-title hlFld-title">Positive Social Support, Negative Social Exchanges, and Suicidal Behavior in College Students</span>
url： https://www.tandfonline.com/doi/full/10.1080/07448481.2010.515635 

raw_title： <span class="NLM_article-title hlFld-title">Editorial Board Page</span>
url： https://www.tandfonline.com/doi/abs/10.1080/01644300.1981.10393044 

raw_title： <span class="NLM_article-title hlFld-title">The Use of Designated Drivers by US College Students: A National Study</span>
url： https://www.tandfonline.com/doi/abs/10.1080/07448489909595640 

raw_title： <span class="NLM_article-title hlFld-title">Dispensing Medications at a Small Health Service: An Alternative to a Dispensing Pharmacist

url： https://www.tandfonline.com/doi/abs/10.1080/07448481.1991.9936240 

raw_title： <span class="NLM_article-title hlFld-title">Psychosocial Correlates of Disordered Eating in Female Collegiate Athletes: Validation of the ATHLETE Questionnaire</span>
url： https://www.tandfonline.com/doi/abs/10.3200/JACH.54.3.149-156 

raw_title： <span class="NLM_article-title hlFld-title">Self-reported Adult Attention-Deficit/Hyperactivity Disorder Symptoms Among College Students</span>
url： https://www.tandfonline.com/doi/full/10.1080/07448481.2010.483718 

raw_title： <span class="NLM_article-title hlFld-title">A Personal Diary of the Outsourcing of a University Student Health Service</span>
url： https://www.tandfonline.com/doi/abs/10.1080/07448480009596280 

raw_title： <span class="NLM_article-title hlFld-title">Development of the Core Alcohol and Drug Survey: Initial Findings and Future Directions</span>
url： https://www.tandfonline.com/doi/abs/10.1080/07448481.1994.9936356 

raw_title： <span class=

url： https://www.tandfonline.com/doi/full/10.1080/07448481.2018.1464335 

raw_title： <span class="NLM_article-title hlFld-title">The Use of Support Groups to Raise Self-Esteem for Women Clients</span>
url： https://www.tandfonline.com/doi/abs/10.1080/07448481.1988.9939046 

raw_title： <span class="NLM_article-title hlFld-title">Recommendations for Institutional Prematriculation Immunizations</span>
url： https://www.tandfonline.com/doi/abs/10.3200/JACH.55.3.126-260 

raw_title： <span class="NLM_article-title hlFld-title">Facilities</span>
url： https://www.tandfonline.com/doi/abs/10.1080/07448481.1984.9936166 

raw_title： <span class="NLM_article-title hlFld-title">Peer Substance Use Associated With the Co-occurrence of Borderline Personality Disorder Features and Drug Use Problems in College Students</span>
url： https://www.tandfonline.com/doi/full/10.1080/07448481.2010.513407 

raw_title： <span class="NLM_article-title hlFld-title">Principles for Assuring the Health of College Students:

# =============
# 二. abstract字段查询
# =============

## 1. 检查abstract数据类型（abstract-Html String）

In [18]:
if 'abstract' in data.columns:
    have_abstract = data[data['abstract'].notnull()]
    abstract_type = []
    abstract_type_warn = 0
    if 'abstract' in data.columns:
        for i in range(len(have_abstract)):
            abstract_type.append(str(isinstance(have_abstract.iloc[i]['abstract'],str)))
            if not isinstance(have_abstract.iloc[i]['abstract'],str):
                abstract_type_warn = abstract_type_warn + 1
        if 'False' not in abstract_type:
            print('正常，abstract数据类型正确')
        else:
            print('异常，abstract数据类型可能存在错误')
            print('【abstract数据类型可能存在错误的数据量 / 存在abstract的数据总量】  <==>  【',abstract_type_warn,'/',len(have_abstract),'】\n')
            for i in range(len(have_abstract)):
                if not isinstance(have_abstract.iloc[i]['abstract'],str):
                    print('出现问题的abstract数据类型：',type(have_abstract.iloc[i]['abstract']))
                    print('url：',have_abstract.iloc[i]['url'][0])
else:
    print('异常，可能缺少必填字段abstract')

正常，abstract数据类型正确


## 2. 检查abstract数据

**(1) 检查 abstract字段缺失的情况**

In [19]:
if 'abstract' in data.columns:
    no_abstract = data[data['abstract'].isnull()]
    if len(no_abstract) == 0:
        print('正常，未发现abstract缺失现象')
    else:
        print('异常，可能存在abstract缺失现象')
        print('【abstract数据类型可能缺失的数据量 / 数据总量】  <==>  【',len(no_abstract),'/',len(data),'】')
        print('请从以下随机抽样检查，查看是否存在抓取遗漏：\n')
        for i in range(len(no_abstract)):
            print('url：',no_abstract.iloc[i]['url'][0])
else:
    print('异常，可能缺少必填字段abstract')

异常，可能存在abstract缺失现象
【abstract数据类型可能缺失的数据量 / 数据总量】  <==>  【 683 / 2484 】
请从以下随机抽样检查，查看是否存在抓取遗漏：

url： https://www.tandfonline.com/doi/abs/10.1080/01644300.1981.10393046
url： https://www.tandfonline.com/doi/full/10.1080/07448481.2010.519014
url： https://www.tandfonline.com/doi/abs/10.1080/01644300.1981.10393028
url： https://www.tandfonline.com/doi/abs/10.1080/01644300.1981.10393064
url： https://www.tandfonline.com/doi/full/10.1080/07448481.2018.1431917
url： https://www.tandfonline.com/doi/abs/10.1080/01644300.1980.10392950
url： https://www.tandfonline.com/doi/abs/10.1080/07448481.1987.9939648
url： https://www.tandfonline.com/doi/abs/10.1080/07448481.1994.9939081
url： https://www.tandfonline.com/doi/abs/10.1080/07448481.1981.9938888
url： https://www.tandfonline.com/doi/abs/10.1080/07448481.1992.9936306
url： https://www.tandfonline.com/doi/abs/10.1080/01644300.1978.10392839
url： https://www.tandfonline.com/doi/abs/10.1080/07448489709595588
url： https://www.tandfonline.com/doi/abs/10.1080/0

**(2) 检查 abstract字段异常的情况**

In [20]:
if 'abstract' in data.columns:
    have_abstract = data[data['abstract'].notnull()]
    tag = 0
    for i in range(len(have_abstract)):
        abstract = have_abstract.iloc[i]['abstract'].strip()
        if len(abstract) != 0:
            if len(abstract) < 400 :
                if tag == 0:
                    tag = 1
                    print('异常，abstract过短，可能存在问题')
                print('abstract : ',abstract)
                print('url：',have_abstract.iloc[i]['url'][0],'\n')
    if tag == 0:
        print('正常，未发现abstract异常情况')
                
else:
    print('异常，可能缺少必填字段abstract')

正常，未发现abstract异常情况


# =============
# 三. keywords、terms字段检查
# =============

## 1. 检查keywords字段

**(1) 检查 keywords字段数据格式（keywords-String）**

In [21]:
if 'keywords' in data.columns:
    have_keywords = data[data['keywords'].notnull()]
    keywords_type = []
    keywords_type_warn = 0
    for i in range(len(have_keywords)):
        keywords_type.append(str(isinstance(have_keywords.iloc[i]['keywords'],str)))
        if not isinstance(have_keywords.iloc[i]['keywords'],str):
            keywords_type_warn = keywords_type_warn + 1
    if 'False' not in keywords_type:
        print('正常，keywords数据类型正确')
    else:
        print('异常，keywords数据类型可能存在错误')
        print('【keywords数据类型可能存在错误的数据量 / 存在keywords的数据总量】  <==>  【',keywords_type_warn,'/',len(have_keywords),'】\n')
        for i in range(len(have_keywords)):
            if not isinstance(have_keywords.iloc[i]['keywords'],str):
                print('出现问题的keywords数据类型：',type(have_keywords.iloc[i]['keywords']))
                print('url：',have_keywords.iloc[i]['url'][0])
else:
    print('异常，可能缺少字段keywords，请详细检查数据源')

正常，keywords数据类型正确


**(2) 检查 keywords字段缺失问题**

In [22]:
if 'keywords' in data.columns:
    no_keywords = data[data['keywords'].isnull()]
    if len(no_keywords) == 0:
        print('正常，未发现keywords缺失现象')
    else:
        print('异常，可能存在keywords缺失现象')
        print('【keywords数据类型可能缺失的数据量 / 数据总量】  <==>  【',len(no_keywords),'/',len(data),'】')
        print('请从以下随机抽样检查，查看是否存在抓取遗漏：\n')
        for i in range(len(no_keywords)):
            print('url：',no_keywords.iloc[i]['url'][0])
else:
    print('异常，可能缺少字段keywords，请详细检查数据源')

异常，可能存在keywords缺失现象
【keywords数据类型可能缺失的数据量 / 数据总量】  <==>  【 1025 / 2484 】
请从以下随机抽样检查，查看是否存在抓取遗漏：

url： https://www.tandfonline.com/doi/abs/10.1080/01644300.1981.10393046
url： https://www.tandfonline.com/doi/abs/10.1080/01644300.1981.10393017
url： https://www.tandfonline.com/doi/full/10.1080/07448481.2010.519014
url： https://www.tandfonline.com/doi/abs/10.1080/01644300.1981.10393028
url： https://www.tandfonline.com/doi/abs/10.1080/01644300.1979.10392875
url： https://www.tandfonline.com/doi/abs/10.1080/01644300.1981.10393064
url： https://www.tandfonline.com/doi/full/10.1080/07448481.2018.1431917
url： https://www.tandfonline.com/doi/abs/10.1080/01644300.1980.10392950
url： https://www.tandfonline.com/doi/abs/10.1080/07448481.1987.9939648
url： https://www.tandfonline.com/doi/abs/10.1080/07448481.1994.9939081
url： https://www.tandfonline.com/doi/abs/10.1080/07448481.1983.9939557
url： https://www.tandfonline.com/doi/abs/10.1080/07448481.1981.9938888
url： https://www.tandfonline.com/doi/abs/10.

url： https://www.tandfonline.com/doi/abs/10.1080/07448481.1990.9936206
url： https://www.tandfonline.com/doi/full/10.1080/07448480903482510
url： https://www.tandfonline.com/doi/abs/10.1080/01644300.1979.10392919
url： https://www.tandfonline.com/doi/abs/10.3200/JACH.54.6.317-326
url： https://www.tandfonline.com/doi/abs/10.1080/07448481.1989.9937490
url： https://www.tandfonline.com/doi/abs/10.1080/07448481.1982.9938907
url： https://www.tandfonline.com/doi/abs/10.1080/07448481.1982.9938923
url： https://www.tandfonline.com/doi/abs/10.1080/01644300.1981.10393010
url： https://www.tandfonline.com/doi/abs/10.1080/07448489909595645
url： https://www.tandfonline.com/doi/full/10.1080/07448481.2017.1341901
url： https://www.tandfonline.com/doi/abs/10.1080/07448480209603452
url： https://www.tandfonline.com/doi/full/10.1080/07448481.2014.1002304
url： https://www.tandfonline.com/doi/abs/10.1080/07448480009596273
url： https://www.tandfonline.com/doi/abs/10.1080/01644300.1981.10393044
url： https://www.tan

**(3) 检查 keywords字段数据问题**

In [23]:
if 'keywords' in data.columns:
    have_keywords = data[data['keywords'].notnull()]
    tag = 0
    for i in range(len(have_keywords)):
        keywords = str(have_keywords.iloc[i]['keywords']).split(';')
        if len(keywords) == 1:
            if keywords[0] == 'nan':
                continue
            else:
                if tag == 0:
                    tag = 1
                    print('异常，keywords字段可能出现问题')
                    print('请详细检查以下keywords是否出现问题：\n')
                print('keywords：',have_keywords.iloc[i]['keywords'])
                print('url：',have_keywords.iloc[i]['url'][0],'\n')
        if (not have_keywords.iloc[i]['keywords'][0].isalnum()) or (not have_keywords.iloc[i]['keywords'][-1].isalnum()) or ('; ' in have_keywords.iloc[i]['keywords']) or ('  ' in have_keywords.iloc[i]['keywords']):
            if tag == 0:
                tag = 1
                print('异常，keywords字段可能出现问题')
                print('请详细检查以下keywords是否出现问题：\n')
            print('keywords：',have_keywords.iloc[i]['keywords'])
            print('url：',have_keywords.iloc[i]['url'][0],'\n')
    if tag == 0:
        print('正常，未发现keywords存在问题')
else:
    print('异常，可能缺少字段keywords，请详细检查数据源')

异常，keywords字段可能出现问题
请详细检查以下keywords是否出现问题：

keywords： alcohol policy;binge drinking;college administrators;historically black colleges and universities (HBCUs)
url： https://www.tandfonline.com/doi/abs/10.3200/JACH.54.1.45-49 

keywords： healthcare reform;insurance;qualified student health plans (QSHPs)
url： https://www.tandfonline.com/doi/abs/10.1080/07448481.1995.9937510 

keywords： acculturation;Filipino students;optimism;positive health practices (PHP)
url： https://www.tandfonline.com/doi/full/10.1080/07448481.2012.753889 

keywords： binge eating;college students;gambling;risk-taking;heavy drinking;South Oaks Gambling Screen (SOGS)
url： https://www.tandfonline.com/doi/abs/10.3200/JACH.52.6.245-256 

keywords： athletes;competitive sports;immunization;infectious diseases;prematriculation immunization requirements (PIRs)
url： https://www.tandfonline.com/doi/abs/10.1080/07448480009596291 

keywords： high-risk drinking;learning collaborative;measurement;model for improvement;Plan–Do–Stud

**(4) 浏览检查 keywords字段**

In [24]:
if "keywords" in data.columns:
    have_keywords = data[data['keywords'].notnull()]
    print('请浏览检查keywords字段：\n')
    for i in range(len(have_keywords)):
        print('keywords：',have_keywords.iloc[i]['keywords'])
        print('url：',have_keywords.iloc[i]['url'][0],'\n')
else:
    print('异常，可能缺少字段keywords，请详细检查数据源')

请浏览检查keywords字段：

keywords： clinical medicine;gender;health education
url： https://www.tandfonline.com/doi/full/10.1080/07448481.2012.750607 

keywords： Campus sexual assault;gender;missing data;response rates;survey methodology
url： https://www.tandfonline.com/doi/full/10.1080/07448481.2018.1462817 

keywords： College students;disordered eating;drunkorexia;high-risk drinking
url： https://www.tandfonline.com/doi/full/10.1080/07448481.2018.1500470 

keywords： athletics;college students;nonmedical prescription drug use
url： https://www.tandfonline.com/doi/abs/10.3200/JACH.57.2.211-220 

keywords： alcohol;college students;mixed methods;responsible drinking
url： https://www.tandfonline.com/doi/full/10.1080/07448481.2010.502196 

keywords： colleges and universities;health center;mission statement;organizational development
url： https://www.tandfonline.com/doi/abs/10.1080/07448481.1995.9939107 

keywords： alcohol;college students;designated driver;drinking and driving;risky behaviors
url： ht

keywords： American College Health Association;college counseling;college mental health;college psychological services;history of college mental health
url： https://www.tandfonline.com/doi/full/10.1080/07448480903297546 

keywords： Alcohol;clinical medicine;community health;health education
url： https://www.tandfonline.com/doi/full/10.1080/07448481.2017.1341767 

keywords： college students;perception;tattoo
url： https://www.tandfonline.com/doi/abs/10.3200/JACH.56.5.593-596 

keywords： alcohol expectancy;cognitive appraisals;sexual risk;condom use
url： https://www.tandfonline.com/doi/full/10.1080/07448481003599112 

keywords： First year students;greek life;history of mental health problems;problem drinking
url： https://www.tandfonline.com/doi/full/10.1080/07448481.2018.1432625 

keywords： college women;sexual assault;sexual coercion;victimization
url： https://www.tandfonline.com/doi/abs/10.1080/07448489809595602 

keywords： alcohol;drug facilitation;incapacitation;rape;sexual assault;sex

url： https://www.tandfonline.com/doi/abs/10.3200/JACH.56.6.701-705 

keywords： alcohol;college students;fraternities;perceptions;social norms
url： https://www.tandfonline.com/doi/abs/10.1080/07448480009596286 

keywords： cigarettes;college students;smoking
url： https://www.tandfonline.com/doi/abs/10.1080/07448489909595657 

keywords： Capability and capacity;college health;college students;higher education;leadership;quality improvement
url： https://www.tandfonline.com/doi/full/10.1080/07448481.2018.1431908 

keywords： college health;independent nursing practice;protocols;standing orders
url： https://www.tandfonline.com/doi/abs/10.1080/07448481.1994.9939082 

keywords： bisexuality;college students;heterosexism;homophobia;homosexuality;lesbians;sexual orientation
url： https://www.tandfonline.com/doi/abs/10.1080/07448481.1994.9938446 

keywords： Freshman 15;qualitative;weight gain
url： https://www.tandfonline.com/doi/full/10.1080/07448481.2011.555931 

keywords： ecological momentary asses

keywords： college students;physical activity;stages of change
url： https://www.tandfonline.com/doi/abs/10.1080/07448481.1995.9937506 

keywords： body mass index;bioelectrical impedance;college athlete;obesity;overweight;waist circumference
url： https://www.tandfonline.com/doi/abs/10.3200/JACH.57.1.33-38 

keywords： commuter school;mental health;risk and protective factors;suicide;suicide ideation
url： https://www.tandfonline.com/doi/abs/10.3200/JACH.55.1.17-26 

keywords： alcohol use;athletics;drinking games;college health
url： https://www.tandfonline.com/doi/abs/10.3200/JACH.56.6.651-656 

keywords： body image;eating disorders;Freshman 15;weight
url： https://www.tandfonline.com/doi/abs/10.1080/07448480209596023 

keywords： alcohol;binge drinking;enabling;fraternities
url： https://www.tandfonline.com/doi/abs/10.1080/07448489909595669 

keywords： alcohol;social norms;reference group;college students and drinking norms
url： https://www.tandfonline.com/doi/full/10.1080/07448481.2010.50219

keywords： feminist theory;gender;health synergies;men's health studies;masculinity;reciprocality;relational theory
url： https://www.tandfonline.com/doi/abs/10.1080/07448480009596295 

keywords： academic medical center;college health;student health center
url： https://www.tandfonline.com/doi/abs/10.3200/JACH.57.3.341-344 

keywords： athlete;papilloma virus;warts;weight lifting
url： https://www.tandfonline.com/doi/abs/10.1080/07448481.1995.9939105 

keywords： nursing protocols;oral contraceptives;patient education;the Pill
url： https://www.tandfonline.com/doi/abs/10.1080/07448481.1995.9940471 

keywords： incest;group psychotherapy;students
url： https://www.tandfonline.com/doi/abs/10.1080/07448481.1990.9936200 

keywords： Anxiety;coping;depression;group;hope;intervention;resilience;stress
url： https://www.tandfonline.com/doi/full/10.1080/07448481.2016.1227826 

keywords： body fat;body shape;college students;freshman 15;lean body mass;weight gain;weight loss
url： https://www.tandfonline.co

## 2. 检查terms字段

**(1) 检查 terms字段数据格式（terms-String）**

In [25]:
if 'terms' in data.columns:
    have_terms = data[data['terms'].notnull()]
    terms_type = []
    terms_type_warn = 0
    for i in range(len(have_terms)):
        terms_type.append(str(isinstance(have_terms.iloc[i]['terms'],str)))
        if not isinstance(have_terms.iloc[i]['terms'],str):
            terms_type_warn = terms_type_warn + 1
    if 'False' not in terms_type:
        print('正常，terms数据类型正确')
    else:
        print('异常，terms数据类型可能存在错误')
        print('【terms数据类型可能存在错误的数据量 / 存在terms的数据总量】  <==>  【',terms_type_warn,'/',len(have_terms),'】\n')
        for i in range(len(have_terms)):
            if not isinstance(have_terms.iloc[i]['terms'],str):
                print('出现问题的terms数据类型：',type(have_terms.iloc[i]['terms']))
                print('url：',have_terms.iloc[i]['url'][0])
else:
    print('异常，可能缺少字段terms，请详细检查数据源')

异常，可能缺少字段terms，请详细检查数据源


**(2) 检查 terms字段缺失问题**

In [26]:
if 'terms' in data.columns:
    no_terms = data[data['terms'].isnull()]
    if len(no_terms) == 0:
        print('正常，未发现terms缺失现象')
    else:
        print('异常，可能存在terms缺失现象')
        print('【terms数据类型可能缺失的数据量 / 数据总量】  <==>  【',len(no_terms),'/',len(data),'】')
        print('请从以下随机抽样检查，查看是否存在抓取遗漏：\n')
        for i in range(len(no_terms)):
            print('url：',no_terms.iloc[i]['url'][0])
else:
    print('异常，可能缺少字段terms，请详细检查数据源')

异常，可能缺少字段terms，请详细检查数据源


**(3) 检查 terms字段数据问题**

In [27]:
if 'terms' in data.columns:
    have_terms = data[data['terms'].notnull()]
    tag = 0
    for i in range(len(have_terms)):
        terms = str(have_terms.iloc[i]['terms']).split(';')
        if len(terms) == 1:
            if terms[0] == 'nan':
                continue
            else:
                if tag == 0:
                    tag = 1
                    print('异常，terms字段可能出现问题')
                    print('请详细检查以下terms是否出现问题：\n')
                print('terms：',have_terms.iloc[i]['terms'])
                print('url：',have_terms.iloc[i]['url'][0],'\n')
        if (not have_terms.iloc[i]['terms'][0].isalnum()) or (not have_terms.iloc[i]['terms'][-1].isalnum()):
            if tag == 0:
                tag = 1
                print('异常，terms字段可能出现问题')
                print('请详细检查以下terms是否出现问题：\n')
            print('terms：',have_terms.iloc[i]['terms'])
            print('url：',have_terms.iloc[i]['url'][0],'\n')
    if tag == 0:
        print('正常，未发现terms存在问题')
else:
    print('异常，可能缺少字段terms，请详细检查数据源')

异常，可能缺少字段terms，请详细检查数据源


**(4) 浏览检查 terms字段**

In [28]:
if "terms" in data.columns:
    have_terms = data[data['terms'].notnull()]
    print('请浏览检查terms字段：\n')
    for i in range(len(have_terms)):
        print('terms：',have_terms.iloc[i]['terms'])
        print('url：',have_terms.iloc[i]['url'][0],'\n')
else:
    print('异常，可能缺少字段terms，请详细检查数据源')

异常，可能缺少字段terms，请详细检查数据源


# =============
# 四. authors字段检查
### 1. 检查authors字段缺失情况
### 2. 检查name字段
### 3. 检查org字段
### 4. 检查email字段
# =============

## 1. 检查authors字段缺失情况

In [29]:
if 'authors' in data.columns:
    no_authors = data[data['authors'].isnull()]
    if len(no_authors) == 0:
        print('正常，未发现authors字段缺失情况')
    else:
        print('异常，可能存在authors缺失现象')
        print('【authors数据类型可能缺失的数据量 / 数据总量】  <==>  【',len(no_authors),'/',len(data),'】')
        print('请从以下随机抽样检查，查看是否存在抓取遗漏：\n')
        for i in range(len(no_authors)):
            print('url：',no_authors.iloc[i]['url'][0])
else:
    print('异常，可能缺少字段authors，请详细检查数据源')

异常，可能存在authors缺失现象
【authors数据类型可能缺失的数据量 / 数据总量】  <==>  【 170 / 2484 】
请从以下随机抽样检查，查看是否存在抓取遗漏：

url： https://www.tandfonline.com/doi/abs/10.1080/01644300.1981.10393064
url： https://www.tandfonline.com/doi/abs/10.1080/01644300.1980.10392950
url： https://www.tandfonline.com/doi/abs/10.1080/07448481.1987.9939648
url： https://www.tandfonline.com/doi/abs/10.1080/07448481.1981.9938888
url： https://www.tandfonline.com/doi/full/10.1080/07448481.2014.992723
url： https://www.tandfonline.com/doi/abs/10.1080/07448481.1985.9938930
url： https://www.tandfonline.com/doi/abs/10.1080/01644300.1981.10393008
url： https://www.tandfonline.com/doi/full/10.1080/07448481.2013.845728
url： https://www.tandfonline.com/doi/abs/10.1080/07448481.1991.9936253
url： https://www.tandfonline.com/doi/abs/10.1080/01644300.1980.10392954
url： https://www.tandfonline.com/doi/abs/10.1080/01644300.1979.10392890
url： https://www.tandfonline.com/doi/full/10.1080/07448481.2011.567186
url： https://www.tandfonline.com/doi/full/10.1080

## 2. 检查pos字段

**(1) 检查 pos字段数据是否缺失**

In [30]:
if 'authors' in data.columns:
    have_authors = data[data['authors'].notnull()]
    tag = 0
    for i in range(len(have_authors)):
        if len(have_authors.iloc[i]['authors']) != 0:
            for n in range(len(have_authors.iloc[i]['authors'])):
                author = have_authors.iloc[i]['authors'][n]
                if 'pos' in author:
                    tag = 0
                else:
                    tag = 1
                    break
    if tag == 0:
        print('正常，未发现authors字段中pos缺失现象')
    else:
        print('异常，authors字段中可能缺少必填字段pos')
else:
    print('异常，可能缺少字段authors，请详细检查数据源')

正常，未发现authors字段中pos缺失现象


**(2) 检查 pos字段数据类型（pos-Int32）**

In [32]:
if 'authors' in data.columns:
    have_authors = data[data['authors'].notnull()]
    tag = 0
    for i in range(len(have_authors)):
        if len(have_authors.iloc[i]['authors']) != 0:
            for n in range(len(have_authors.iloc[i]['authors'])):
                author = have_authors.iloc[i]['authors'][n]
                if 'pos' in author:
                    pos = author['pos']
                    if not isinstance(pos,int):
                        tag = 1
                        break
    if tag == 0:
        print('正常，未发现authors字段中pos数据格式问题')
    else:
        print('异常，authors字段中pos字段格式可能出现错误')
        print('要求格式为Int32，实际格式为',type(pos))
else:
    print('异常，可能缺少字段authors，请详细检查数据源')

正常，未发现authors字段中pos数据格式问题


**(3) 检查 pos字段数据问题**

In [31]:
if 'authors' in data.columns:
    have_authors = data[data['authors'].notnull()]
    tag = 0
    for i in range(len(have_authors)):
        if len(have_authors.iloc[i]['authors']) != 0:
            for n in range(len(have_authors.iloc[i]['authors'])):
                author = have_authors.iloc[i]['authors'][n]
                if 'pos' in author:
                    pos = author['pos']
                    if isinstance(pos,int):
                        if pos != n:
                            tag = 1
                            break
    if tag == 0:
        print('正常，未发现authors字段中pos数据问题')
    else:
        print('异常，authors字段中pos字段数据可能出现错误')
        print('pos字段应该从 0 开始')
else:
    print('异常，可能缺少字段authors，请详细检查数据源')

正常，未发现authors字段中pos数据问题


## 3. 检查name字段

**(1) 检查 name字段数据问题**

In [32]:
if 'authors' in data.columns:
    have_authors = data[data['authors'].notnull()]
    tag = 0
    for i in range(len(have_authors)):
        if len(have_authors.iloc[i]['authors']) != 0:
            for n in range(len(have_authors.iloc[i]['authors'])):
                authors = have_authors.iloc[i]['authors'][n]
                if 'name' in authors:
                    namestr = authors['name']
                    if namestr :
                        if ('\n' in namestr) or ('\t' in namestr) or ('\r' in namestr):
                            tag = 1
                        if (len(namestr) <= 3) or (len(namestr) > 30) or (' and ' in namestr):
                            tag = 1
                        if ('(' in namestr) or (not namestr[0].isalpha()) or (not namestr[-1].isalpha()):
                            if (namestr[0] == ' ') or (namestr[-1] == ' '):
                                tag = 1
                            elif namestr[-1] != '.':
                                tag = 1
                    else:
                        tag = 1
    if tag == 0:
        print('正常，未发现name存在问题')
    else:
        print('异常，name字段可能出现问题')
        print('请详细检查以下name是否出现问题：\n')
    
        for i in range(len(have_authors)):
            if len(have_authors.iloc[i]['authors']) != 0:
                for n in range(len(have_authors.iloc[i]['authors'])):
                    authors = have_authors.iloc[i]['authors'][n]
                    if 'name' in authors:
                        namestr = authors['name']
                        if namestr :
                            if ('\n' in namestr) or ('\t' in namestr) or ('\r' in namestr):
                                print('name中可能存在转义字符 ntr')
                                print('name：',namestr)
                                print('url：',have_authors.iloc[i]['url'][0],'\n')
                            if (len(namestr) <= 3) or (len(namestr) > 30) or (' and ' in namestr):
                                print('name太短或太长，可能有问题')
                                print('name：',namestr)
                                print('url：',have_authors.iloc[i]['url'][0],'\n')
                            if ('(' in namestr) or (not namestr[0].isalpha()) or (not namestr[-1].isalpha()):
                                if namestr.strip() == '':
                                    print('name可能为空')
                                    print('name：',namestr)
                                    print('url：',have_authors.iloc[i]['url'][0],'\n')
                                elif (namestr[0] == ' ') or (namestr[-1] == ' '):
                                    print('name两侧可能存在空格')
                                    print('name：',namestr)
                                    print('url：',have_authors.iloc[i]['url'][0],'\n')
                                elif namestr[-1] != '.':
                                    print('name中可能存在异常')
                                    print('name：',namestr)
                                    print('url：',have_authors.iloc[i]['url'][0],'\n')
                        else:
                                print('name为空')
                                print('name：',namestr)
                                print('url：',have_authors.iloc[i]['url'][0],'\n')
else:
    print('异常，可能缺少字段authors，请详细检查数据源')

异常，name字段可能出现问题
请详细检查以下name是否出现问题：

name太短或太长，可能有问题
name： Members of the Student Health Services at Academic Medical Centers Task Force
url： https://www.tandfonline.com/doi/abs/10.1080/07448480109596015 

name太短或太长，可能有问题
name： Maria Lilian Coelho de Oliveira
url： https://www.tandfonline.com/doi/abs/10.3200/JACH.57.3.367-372 

name中可能存在异常
name： Chudley (Chad) E. Werch
url： https://www.tandfonline.com/doi/full/10.1080/07448481.2011.574765 

name中可能存在异常
name： Tsung-Chieh (Jane) Fu
url： https://www.tandfonline.com/doi/full/10.1080/07448481.2018.1462827 



**(2) 浏览检查 name字段**

In [33]:
if 'authors' in data.columns:
    have_authors = data[data['authors'].notnull()]
    for i in range(len(have_authors)):
        if len(have_authors) != 0 :
            for n in range(len(have_authors.iloc[i]['authors'])):
                author = have_authors.iloc[i]['authors'][n]
                if 'name' in author:
                    namestr = author['name']
                    print('name：',namestr)
                    # 下面一行代码用于显示url，有问题时候可取消注释
#                     print('url : ',have_authors.iloc[i]['url'][0],'\n')
else:
    print('异常，可能缺少字段authors，请详细检查数据源')

name： Lisa L. Lindley
name： Julia S. Elkind
name： Suzanne N. Landi BA
name： Heather M. Brandt
name： Molly Rosenberg
name： Ashley Townes
name： Shaneil Taylor
name： Maya Luetke
name： Debby Herbenick
name： Harold L. Enarson
name： Carly Thompson-Memmer
name： Tavis Glassman
name： Aaron Diehr
name： Ikponmwosa Owie
name： Jason A. Ford
name： Adam E. Barry
name： Patricia Goodson
name： Colin Ramirez
name： James Turner
name： Joseph F. Chance
name： Mark J. Warner
name： Sarah S. Dermody
name： JeeWon Cheong
name： Christine A. Walther
name： James B. McClenahan
name： Janet L. Thomas
name： Larry An
name： Xianghua Luo
name： Robyn M. Scherber BS
name： Carla J. Berg
name： Dave Golden BA
name： Edward P. Ehlinger
name： Sharon E. Murphy
name： Stephen S. Hecht
name： Jasjit S. Ahluwalia
name： Eileen Schattner
name： Kari Jo Harris
name： Julee N. Stearns
name： Rachel G. Kovach
name： Solomon W. Harrar
name： Kaleea R. Lewis
name： Alyssa Robillard
name： Deborah Billings
name： Kellee White
name： Erica Weintraub Aust

name： Kimberly M. Caldeira
name： Kathryn B. Vincent
name： Kevin E. O’Grady
name： Amelia M. Arria
name： Lindsay A. Taliaferro
name： Barbara A. Rienzo
name： R. Morgan Pigg
name： M. David Miller
name： Virginia J. Dodd
name： Timo Niemi
name： J. Stephen McDaniel
name： Lisa M. Carlson
name： Nancy J. Thompson
name： David W. Purcell JD
name： Robert F. Valois
name： Kathryn A. Waring
name： Thomas V. Foster Jr.
name： John M. Goldenring
name： Tobias Reynolds-Tylus
name： Kaylee M. Lukacena
name： Brian L. Quick
name： Bruce Simons-Morton
name： Denise Haynie
name： Fearghal O'Brien
name： Leah Lipsky
name： Joe Bible
name： Danping Liu
name： Ali M. Shropshire
name： Renee Brent-Hotchkiss
name： Urkovia K. Andrews
name： Brett N. Steenbarger
name： Jocelyn Mullins DVM
name： Robert Cook
name： Charles Rinaldo
name： Eric Yablonsky
name： Rachel Hess
name： Paolo Piazza
name： Shana M. Wilson
name： Katherine E. Darling
name： Amy J. Fahrenkamp
name： Alexandra L. D’Auria
name： Amy F. Sato
name： Joshua E. Kaplan
name： M

name： Ann M. Craun
name： Jerry L. Deffenbacher
name： Laura M. Miller
name： Shirley P. Fisher
name： Michele M. Fisher DPE
name： Emil Chiauzzi
name： Elizabeth Donovan
name： Ryan Black
name： Elizabeth Cooney
name： Allison Buechner BA
name： Mollie Wood
name： Stacy Andes
name： Kathryn Poet
name： Stephen McWilliams
name： Alan R. Hinman
name： Stephen R. Preblud
name： Susan McConville Hidalgo
name： Edward I. Lifshitz
name： Jeremiah Weinstock
name： James P. Whelan
name： Andrew Meyers
name： Stephen M. Blom
name： Sarah W. Whitton
name： Eliza M. Weitbrecht BA
name： Amanda D. Kuryluk
name： Michael R. Bruner BS
name： Courtney Stevens
name： Cindy H. Liu
name： Justin A. Chen
name： Robert W. Gage JACHA
name： Belinda J. McIntosh
name： Michael T. Compton
name： Benjamin G. Druss
name： Ruth A. Hopkins
name： Karen M. Demartin
name： Clifford B. Reifler
name： Kathryn A. Stefos
name： Jose V. Nable
name： Carol S. Johnston
name： R. Elizabeth Solomon
name： Corinne Corte BS
name： Robert Wm. Amler
name： Walter A. O

name： Judith R. Cowan
name： Rolffs S. Pinkerton
name： Edward F. Etzel
name： W. J. Kenneth Rockwell
name： Joseph E. Talley
name： Jane Clark Moorman
name： Amy Thompson
name： James H. Price
name： Adam J. Mrdjenovich
name： Jagdish Khubchandani MBBS
name： Mary Jo Coiro
name： Alexandra H. Bettis
name： Bruce E. Compas
name： Walter W. Williams
name： Daniel M. Sosin
name： Karen M. Kaplan
name： Bradley S. Hersh
name： Stephen R. Preblud
name： Kimberly C. Roberts
name： Sharon Danoff-Burg
name： Patience Wilson Cameron
name： F. C. L. Allen M.Sc.
name： J. B. Cole
name： Christina M. Lewis
name： Andris Ziemelis
name： Ronald B. Bucknam
name： Abdulaziz M. Elfessi
name： Corinne L. McNamara
name： Dorothy F. Marsil
name： Christine G. Zimmer
name： Rimsha Ahmed
name： John T. P. Hustad
name： Linda LaSalle
name： Brian Borsari
name： Anna Cantor
name： Catriona Hippman
name： Laura Hercher
name： Jehannine C. Austin
name： Stephen L. Beckley BA
name： Ted W. Grace
name： Richard P. Keeling
name： Marci Loiselle
name： Wa

name： Brittany L. Stevenson
name： Emily M. Sargent
name： Gary Margolis
name： Henry Wechsler
name： Nell H. Gottlieb
name： Douglas Sloan
name： Demetrius A. Abshire
name： Gia Mudd-Martin
name： Debra K. Moser
name： Terry A. Lennie
name： Mary Lou Johnson
name： Bruce E. Donovan
name： Dianne L. Kerr
name： Laura Santurri
name： Patricia Peters
name： Michael Hennessy
name： Brigitte Manteuffel
name： Colleen Dilorio
name： Dan Adame
name： Sonjia Kenya
name： Mitchell Brodsky
name： William Divale
name： John P. Allegrante
name： Robert E. Fullilove
name： Adelaide Getty
name： Helen M. Bannan
name： James W. Dilley
name： John P. Hickenbottom
name： Raymond P. Bissonette
name： Robert M. O'Shea
name： Marilyn M. Buck R. N.
name： Ed. D.
name： Philip W. Meilman
name： Jeffrey R. Cashin
name： Jack McKillip
name： Cheryl A. Presley
name： Hannah J. Solky
name： James E. McKeever
name： Richard A. Perlmutter
name： Thomas E. Gift
name： Robert Fogerty Ba
name： Scott Vierke
name： Steven R. Hawks
name： Hala Madanat
name： T

...

name： Henry Wechsler
name： Mary Rohman
name： Leonard Solomon
name： Marjeanne Collins
name： Gary Fleisher
name： Janet Kreisberg
name： Samuel Fager
name： Emily Rothman ScD
name： Jay Silverman
name： Chizimuzo T. C. Okoli
name： Mary Kay Rayens
name： Amanda T. Wiggins
name： Melinda J. Ickes
name： Karen M. Butler
name： Ellen J. Hahn
name： Brittni D. Jones
name： Renee M. Cunningham-Williams
name： Brett N. Steenbarger
name： Ralph A. Manchester
name： Nancy J. Bell
name： Kirti Kanitkar
name： Kimberly A. Kerksiek
name： Wendy Watson
name： Anindita Das
name： Erin Kostina-Ritchey
name： Matthew H. Russell
name： Kitty Harris
name： Fernando L. Vázquez
name： Vanessa Blanco
name： James W. Brasseur
name： David M. Kaplan
name： George A. Kelley DA
name： Larry Lowing
name： Kristi Kelley MEd
name： Gary Margolis
name： Sabina  White
name： Yong S. Park
name： Tania Israel
name： Elizabeth D. Cordero
name： John K. Scheer
name： Deborah Loper MEd
name： Loree Wagner BS
name： Steven Wise
name： Eric Heiligenstein
name： 

name： Madeline B. Benz
name： Mary Beth Miller
name： Jennifer E. Merrill
name： Kate B. Carey
name： Gerald C. Hyner
name： Christopher L. Melby M. P. H.
name： DHSc.
name： Chia-Yen Tsai
name： Sharon L. Hoerr RD
name： Won O. Song RD
name： Hayden L. Holmes
name： Lauren E. Brewer
name： Stacey A. Kerr
name： Maria Cooper
name： Alexandra Loukas
name： Melissa B. Harrell
name： Cheryl L. Perry
name： Mark E. Giesecke
name： Stephanie L. Lochbihler BA
name： Daniel A. Miller
name： Paul E. Etcheverry
name： James W. Dilley
name： Julie Tinklenberg
name： Bina Patel
name： Kathryn Gelman
name： Ronald Albucher
name： Diane Norton
name： Nell H. Gottlieb
name： Winston C. Liao
name： Benjamin F. Banahan  III
name： P. Anthony McDaniel M.H.C.A.
name： Alan B. Mckay
name： Ian B Marshall MB, BS, MRCGP
name： Matthew P. Martens
name： Tracey L. Rocha BA
name： M. Dolores Cimini
name： Angelina Diaz-Myers
name： Estela M. Rivero
name： Edelgard Wulfert
name： Joelle Monaghan
name： Audrey Steenbeek
name： Erna Snelgrove-Clarke
na

name： Margaret J. Nellis
name： Philip G. Stephan S.T.M.
name： Denyce S. Ford
name： Carolyn R. Goode
name： Philip W. Meilman
name： Jami S. Leichliter
name： Cheryl A. Presley
name： Jessica A. Griffin
name： M. Renée Umstattd
name： Stuart L. Usdan
name： Laura A. Rhodes PharmD
name： Ruth A. Nice
name： Stephen L. Dilts
name： Robert M. House Jr.
name： Wallace R. Arthur
name： Mary E. Hurley
name： Jameson K. Hirsch
name： Alison L. Barton
name： William DeJong
name： Jay A. Winsten
name： Malcolm L. McCutcheon
name： James L. Anderson
name： Robert Carlson
name： John M. Dorman
name： Jan Lahr
name： Carol Moore
name： Jenna D. Anding
name： Richard R. Suminski
name： Linda Boss BS
name： James C. Turner
name： Dominic Cittadino
name： Frederic Morgan Ed.D.
name： William DeJong
name： Linda G. McWhorter
name： Virginia Gil-Rivas
name： Kathleen N. Adamle
name： Tracy A. Riley
name： Tracey Carlson
name： Janet L. Thomas
name： Tracy A. Gerber
name： Tabetha A. Brockman
name： Christi A. Patten
name： Darrell R. Schroed

name： Jichuan Wang
name： Laura Griffin
name： Joyce L. Kee
name： Linda Waters
name： Ariane V. Hollub
name： Michael Reece
name： Debby Herbenick
name： Devon J. Hensel
name： Susan E. Middlestadt
name： Allyson L. Dir
name： Elizabeth N. Riley
name： Melissa A. Cyders
name： Gregory T. Smith
name： Corrie Barnett Struble
name： Lisa L. Lindley
name： Kara Montgomery
name： James Hardin
name： Michelle Burcin
name： Jeffrey A. Hoffman
name： Bahr Weiss
name： Alfred B. Heilburn Jr.
name： Amy Chefitz B.A.
name： Karen A. Kemper
name： Ralph S. Welsh
name： Max Costin
name： Kathryn Graff Low
name： Jiun-Hau Huang SM, ScD
name： William DeJong
name： Laura Gomberg Towvim
name： Shari Kessel Schneider
name： Richard P. Keeling
name： Samir Ranjit Jani
name： Darryl Brown
name： Zekarias Berhane
name： Nadja Peter
name： Susan Solecki
name： Renee Turchi
name： Eric Heiligenstein
name： Michael Gordon
name： Barbara Estabrook
name： Ruth Fessenden
name： Mary Dumas
name： Thomas C. McBride
name： Margaret W. Bridwell
name： Steve

name： Amy Hequembourg
name： Nicole Aydt Klein
name： Michael G. Dudley
name： Gerrit I. van Schalkwyk
name： Chad Beyer
name： Andrés Martin
name： Fred R. Volkmar
name： Paul B. Pedersen
name： Jonathan A. Gur-Yosef
name： Gary S. Leske
name： Louis W. Ripa
name： Taejin Jung
name： Gary R. Heald
name： Alan D. Berkowitz
name： H. Wesley Perkins
name： Isha W. Metzger
name： Claire Blevins
name： Casey D. Calhoun
name： Tiarney D. Ritchwood
name： Amanda K. Gilmore
name： Regan Stewart
name： Kaitlin E. Bountress
name： Gilbert Quintero
name： Dr Mary Alice Gillispie
name： Lynne Ebner
name： Rachel Johnson MT
name： Angela M. Kempf
name： Myra L. Strother
name： Chaoyang Li
name： Harsohena Kaur
name： Terry T-K. Huang
name： Nicole L. Mihalopoulos
name： Peggy Auinger
name： Jonathan D. Klein
name： Jennifer Morrone Joseph
name： Richard A. Perlmutter
name： Allan J. Schwartz
name： Clifford B. Reifler
name： H. Spencer Turner
name： Walton R. Garner
name： Richard R. Lanese
name： Kevin J. Edwards
name： Paul J. Hershberg

## 4. 检查org字段

**(1) 检查 org字段缺失问题**

In [34]:
if 'authors' in data.columns:
    have_authors = data[data['authors'].notnull()]
    tag = 0
    for i in range(len(have_authors)):
        if len(have_authors.iloc[i]['authors']) != 0:
            for n in range(len(have_authors.iloc[i]['authors'])):
                authors = have_authors.iloc[i]['authors'][n]
                if 'org' not in authors:
                    tag = 1
    if tag == 0:
        print('正常，未发现org字段缺失问题')
    else:
        print('异常，org字段可能出现缺失问题')
        for i in range(len(have_authors)):
            if len(have_authors.iloc[i]['authors']) != 0:
                for n in range(len(have_authors.iloc[i]['authors'])):
                    authors = have_authors.iloc[i]['authors'][n]
                    if 'org' not in authors:
                        print('name：',authors['name'])
                        print('url：',have_authors.iloc[i]['url'][0],'\n')
else:
    print('异常，可能缺少字段authors，请详细检查数据源')

异常，org字段可能出现缺失问题
name： Mark J. Warner
url： https://www.tandfonline.com/doi/abs/10.1080/07448481.1995.9939107 

name： James B. McClenahan
url： https://www.tandfonline.com/doi/abs/10.1080/01644300.1981.10393028 

name： Sandra Bygrave Dozier
url： https://www.tandfonline.com/doi/abs/10.1080/07448480109596018 

name： Mary Ann Magaldi
url： https://www.tandfonline.com/doi/abs/10.1080/07448480109596018 

name： Ellen Kresse
url： https://www.tandfonline.com/doi/abs/10.1080/07448480109596018 

name： Eric Heiligenstein
url： https://www.tandfonline.com/doi/abs/10.1080/07448489809596003 

name： Greta Guenther
url： https://www.tandfonline.com/doi/abs/10.1080/07448489809596003 

name： Emil Chiauzzi
url： https://www.tandfonline.com/doi/abs/10.3200/JACH.53.6.263-274 

name： Traci Craig Green Msc
url： https://www.tandfonline.com/doi/abs/10.3200/JACH.53.6.263-274 

name： Sarah Lord
url： https://www.tandfonline.com/doi/abs/10.3200/JACH.53.6.263-274 

name： Christina Thum Ma
url： https://www.tandfonline.com

name： Adam E. Barry
url： https://www.tandfonline.com/doi/full/10.1080/07448481.2015.1072777 

name： Robin G. Sawyer
url： https://www.tandfonline.com/doi/abs/10.1080/07448489809600226 

name： Paul J. Pinciaro
url： https://www.tandfonline.com/doi/abs/10.1080/07448489809600226 

name： Anne Anderson-Sawyer
url： https://www.tandfonline.com/doi/abs/10.1080/07448489809600226 

name： Lisa M. Lewis
url： https://www.tandfonline.com/doi/abs/10.1080/07448480009596278 

name： Richard S. Melton
url： https://www.tandfonline.com/doi/abs/10.1080/07448480009596278 

name： Paul A. Succop
url： https://www.tandfonline.com/doi/abs/10.1080/07448480009596278 

name： Susan L. Rosenthal
url： https://www.tandfonline.com/doi/abs/10.1080/07448480009596278 

name： Desideria Shipp Hacker
url： https://www.tandfonline.com/doi/abs/10.1080/07448481.1993.9940824 

name： Deanna Kraus-Zeilmann PsyD
url： https://www.tandfonline.com/doi/abs/10.1080/07448481.1993.9940824 

name： Bonnie Chakravorty
url： https://www.tandfonline

name： H. Spencer Turner
url： https://www.tandfonline.com/doi/abs/10.1080/01644300.1980.10392934 

name： Arthur S. Rosecan
url： https://www.tandfonline.com/doi/abs/10.1080/07448481.1992.9936301 

name： Teri Aronowitz
url： https://www.tandfonline.com/doi/full/10.1080/07448481.2018.1502994 

name： Lawrence Neinstein
url： https://www.tandfonline.com/doi/abs/10.1080/07448489809595636 

name： Francine Harvey DLM, MBA
url： https://www.tandfonline.com/doi/abs/10.1080/07448489809595636 

name： Rama Ganguly
url： https://www.tandfonline.com/doi/abs/10.1080/07448489809595608 

name： Phillip J. Marty
url： https://www.tandfonline.com/doi/abs/10.1080/07448489809595608 

name： Arthur H. Herold
url： https://www.tandfonline.com/doi/abs/10.1080/07448489809595608 

name： Maria Anderson
url： https://www.tandfonline.com/doi/abs/10.1080/07448489809595608 

name： Tawnya D. O'Keefe
url： https://www.tandfonline.com/doi/abs/10.1080/07448489809595633 

name： Susan J. Henly
url： https://www.tandfonline.com/doi/abs

url： https://www.tandfonline.com/doi/abs/10.1080/07448489909595659 

name： Rama Ganguly
url： https://www.tandfonline.com/doi/abs/10.1080/07448489909595649 

name： Douglas A. Holt
url： https://www.tandfonline.com/doi/abs/10.1080/07448489909595649 

name： John T. Sinnott
url： https://www.tandfonline.com/doi/abs/10.1080/07448489909595649 

name： Henry Wechsler
url： https://www.tandfonline.com/doi/abs/10.1080/07448480109596305 

name： Kathleen Kelley MBA
url： https://www.tandfonline.com/doi/abs/10.1080/07448480109596305 

name： Mark Seibring BA/BS
url： https://www.tandfonline.com/doi/abs/10.1080/07448480109596305 

name： Meichun Kuo ScD
url： https://www.tandfonline.com/doi/abs/10.1080/07448480109596305 

name： Nancy A. Rigotti
url： https://www.tandfonline.com/doi/abs/10.1080/07448480109596305 

name： Colleen A. Carter
url： https://www.tandfonline.com/doi/abs/10.1080/07448480009596286 

name： William M. Kahnweiler
url： https://www.tandfonline.com/doi/abs/10.1080/07448480009596286 

name： Pa

url： https://www.tandfonline.com/doi/abs/10.1080/07448489709595591 

name： Ruth P. Saunders
url： https://www.tandfonline.com/doi/abs/10.1080/07448489709595591 

name： J. Wanzer Drane PE
url： https://www.tandfonline.com/doi/abs/10.1080/07448489709595591 

name： Caroline A. Macera
url： https://www.tandfonline.com/doi/abs/10.1080/07448489709595591 

name： Edward P. Sarafino
url： https://www.tandfonline.com/doi/abs/10.1080/07448489909595677 

name： Maureen Ewing BA
url： https://www.tandfonline.com/doi/abs/10.1080/07448489909595677 

name： Kathy E. Fischer
url： https://www.tandfonline.com/doi/abs/10.1080/07448480009595690 

name： Mark Kittleson
url： https://www.tandfonline.com/doi/abs/10.1080/07448480009595690 

name： Roberta Ogletree
url： https://www.tandfonline.com/doi/abs/10.1080/07448480009595690 

name： Kathleen Welshimer
url： https://www.tandfonline.com/doi/abs/10.1080/07448480009595690 

name： Paula Woehlke
url： https://www.tandfonline.com/doi/abs/10.1080/07448480009595690 

name： Jo

name： Charles R. Baffi
url： https://www.tandfonline.com/doi/abs/10.1080/07448481.1989.9938432 

name： Kelli Kenison Schroeder
url： https://www.tandfonline.com/doi/abs/10.1080/07448481.1989.9938432 

name： Kerry J. Redican
url： https://www.tandfonline.com/doi/abs/10.1080/07448481.1989.9938432 

name： Lawrence McCluskey
url： https://www.tandfonline.com/doi/abs/10.1080/07448481.1989.9938432 

name： Kathryn G. Low
url： https://www.tandfonline.com/doi/abs/10.1080/07448489809595634 

name： Joshua M. Feissner BA
url： https://www.tandfonline.com/doi/abs/10.1080/07448489809595634 

name： Ashley N. Junghans-Rutelonis
url： https://www.tandfonline.com/doi/full/10.1080/07448481.2015.1057146 

name： Kristina I. Suorsa
url： https://www.tandfonline.com/doi/full/10.1080/07448481.2015.1057146 

name： Alayna P. Tackett
url： https://www.tandfonline.com/doi/full/10.1080/07448481.2015.1057146 

name： Edward Burkley
url： https://www.tandfonline.com/doi/full/10.1080/07448481.2015.1057146 

name： John M. Chane

url： https://www.tandfonline.com/doi/abs/10.1080/07448480009596292 

name： Russell D. Ravert
url： https://www.tandfonline.com/doi/full/10.1080/07448481.2015.1015026 

name： Suzanne A. Boren
url： https://www.tandfonline.com/doi/full/10.1080/07448481.2015.1015026 

name： Emily Wiebke
url： https://www.tandfonline.com/doi/full/10.1080/07448481.2015.1015026 

name： Peter S Holck Md, Mph
url： https://www.tandfonline.com/doi/abs/10.3200/JACH.53.4.159-166 

name： Diana Bernstein
url： https://www.tandfonline.com/doi/abs/10.1080/07448481.1988.9939039 

name： Cathy Jo Lore
url： https://www.tandfonline.com/doi/abs/10.1080/07448489709595586 

name： Willard Dalrymple
url： https://www.tandfonline.com/doi/abs/10.1080/07448481.1995.9940910 

name： David A. Birch
url： https://www.tandfonline.com/doi/abs/10.1080/07448489709595600 

name： Terrence P. O'Toole MEd
url： https://www.tandfonline.com/doi/abs/10.1080/07448489709595600 

name： Andrew J. Kanu
url： https://www.tandfonline.com/doi/abs/10.1080/074484

name： Marjorie Whittaker Leidig
url： https://www.tandfonline.com/doi/abs/10.1080/07448481.1992.9936276 

name： Cheryl A. Presley
url： https://www.tandfonline.com/doi/abs/10.1080/07448481.1997.9936885 

name： Jeffrey R. Cashin
url： https://www.tandfonline.com/doi/abs/10.1080/07448481.1997.9936885 

name： Nigel Paneth
url： https://www.tandfonline.com/doi/abs/10.1080/07448480009596276 

name： Eric J. Kort BA
url： https://www.tandfonline.com/doi/abs/10.1080/07448480009596276 

name： Dennis Jurczak
url： https://www.tandfonline.com/doi/abs/10.1080/07448480009596276 

name： Daniel A. Havlichek Jr
url： https://www.tandfonline.com/doi/abs/10.1080/07448480009596276 

name： Kathi Braunlich MBA
url： https://www.tandfonline.com/doi/abs/10.1080/07448480009596276 

name： Glynda Moorer
url： https://www.tandfonline.com/doi/abs/10.1080/07448480009596276 

name： Doreen Vanderjagt MBA
url： https://www.tandfonline.com/doi/abs/10.1080/07448480009596276 

name： Dean Sienko
url： https://www.tandfonline.com/do

**(2) 检查 org字段数据问题**

In [35]:
if 'authors' in data.columns:
    have_authors = data[data['authors'].notnull()]
    tag = 0
    for i in range(len(have_authors)):
        if len(have_authors.iloc[i]['authors']) != 0:
            for n in range(len(have_authors.iloc[i]['authors'])):
                authors = have_authors.iloc[i]['authors'][n]
                if 'org' in authors:
                    orglist = authors['org']
                    if len(orglist) != 0:
                        for t in range(len(orglist)):
                            if len(orglist[t]) != 0:
                                if ('\n' in orglist[t]) or ('\t' in orglist[t]) or ('\r' in orglist[t]) or ('  ' in orglist[t]):
                                    tag = 1
                                if (not orglist[t][0].isalnum()) or (not orglist[t][-1].isalnum()):
                                    if orglist[t][-1] != '.' and orglist[t][-1] != ')':
                                        tag = 1
                                if '@' in orglist[t]:
                                    tag = 1
                            else:
                                tag = 1
                    else:
                        tag = 1

    if tag == 0:
        print('正常，未发现org存在问题')
    else:
        print('异常，org字段可能出现问题')
        print('请详细检查以下org是否出现问题：\n')
    
        for i in range(len(have_authors)):
            if len(have_authors.iloc[i]['authors']) != 0:
                for n in range(len(have_authors.iloc[i]['authors'])):
                    authors = have_authors.iloc[i]['authors'][n]
                    if 'org' in authors:
                        orglist = authors['org']
                        if len(orglist) != 0:
                            for t in range(len(orglist)):
                                if len(orglist[t]) != 0:
                                    if ('\n' in orglist[t]) or ('\t' in orglist[t]) or ('\r' in orglist[t]):
                                        print('org中可能存在转义字符')
                                        print('org：',orglist[t])
                                        print('url：',have_authors.iloc[i]['url'][0],'\n')
                                    if (not orglist[t][0].isalnum()) or (not orglist[t][-1].isalnum()):
                                        if orglist[t][-1] != '.' and orglist[t][-1] != ')':
                                            print('org中可能存在异常')
                                            print('org：',orglist[t])
                                            print('url：',have_authors.iloc[i]['url'][0],'\n')
                                    if '@' in orglist[t]:
                                        print('org中可能存在异常')
                                        print('org：',orglist[t])
                                        print('url：',have_authors.iloc[i]['url'][0],'\n')
                                    if '  ' in orglist[t]:
                                        print('org中可能存在多余空格')
                                        print('org：',orglist[t])
                                        print('url：',have_authors.iloc[i]['url'][0],'\n')
                                else:
                                    print('org中可能为空')
                                    print('org：',orglist)
                                    print('url：',have_authors.iloc[i]['url'][0],'\n')
                        else:
                            print('org中可能为空')
                            print('org：',orglist)
                            print('url：',have_authors.iloc[i]['url'][0],'\n')
else:
    print('异常，可能缺少字段authors，请详细检查数据源')

正常，未发现org存在问题


**(3) 浏览检查 org字段**

In [36]:
if 'authors' in data.columns:
    have_authors = data[data['authors'].notnull()]
    tag = 0
    for i in range(len(have_authors)):
        if len(have_authors) != 0 :
            for n in range(len(have_authors.iloc[i]['authors'])):
                author = have_authors.iloc[i]['authors'][n]
                if 'org' in author:
                    if tag == 0:
                        tag = 1
                    orgstr = author['org']
                    print('org：',orgstr[0])
                    # 下面一行代码用于显示url，有问题时候可取消注释
#                     print('url : ',have_authors.iloc[i]['url'][0],'\n')
    if tag == 0:
        print('异常，可能缺少字段org，请详细检查数据源')
else:
    print('异常，可能缺少字段authors，请详细检查数据源')

org： Department of Global and Community Health , George Mason University , Fairfax , Virginia
org： Department of Global and Community Health , George Mason University , Fairfax , Virginia
org： Department of Global and Community Health , George Mason University , Fairfax , Virginia
org： Department of Health Promotion, Education, and Behavior and the Statewide Cancer Prevention and Control Program, Arnold School of Public Health , University of South Carolina , Columbia , South Carolina
org： Department of Epidemiology, Indiana University School of Public Health-Bloomington, Bloomington, Indiana, USA
org： Department of Applied Health Science, Indiana University School of Public Health-Bloomington, Bloomington, Indiana, USA
org： Department of Applied Health Science, Indiana University School of Public Health-Bloomington, Bloomington, Indiana, USA
org： Department of Epidemiology, Indiana University School of Public Health-Bloomington, Bloomington, Indiana, USA
org： Department of Applied Hea

org： Department of Communication, University of Illinois at Urbana-Champaign, Champaign, Illinois, USA
org： Health Behavior Branch, Division of Intramural Population Health Research, Eunice Kennedy Shriver National Institute of Child Health and Human Development, Bethesda, Maryland, USA
org： Health Behavior Branch, Division of Intramural Population Health Research, Eunice Kennedy Shriver National Institute of Child Health and Human Development, Bethesda, Maryland, USA
org： Health Behavior Branch, Division of Intramural Population Health Research, Eunice Kennedy Shriver National Institute of Child Health and Human Development, Bethesda, Maryland, USA
org： Health Behavior Branch, Division of Intramural Population Health Research, Eunice Kennedy Shriver National Institute of Child Health and Human Development, Bethesda, Maryland, USA
org： Biostatistics and Bioinformatics Branch, Division of Intramural Population Health Research, Eunice Kennedy Shriver National Institute of Child Health an

org： Department of Psychology , University of Cincinnati , Cincinnati , Ohio
org： Department of Psychology , University of Cincinnati , Cincinnati , Ohio
org： Department of Psychology , University of Cincinnati , Cincinnati , Ohio
org： Department of Psychology , University of Cincinnati , Cincinnati , Ohio
org： Willamette University, Salem, Oregon, USA
org： Beth Israel Deaconess Medical School, Harvard Medical School, Boston, Massachusetts, USA
org： Massachusetts General Hospital, Harvard Medical School, Boston, Massachusetts, USA
org： Department of Psychiatry and Behavioral Sciences , Emory University and Psychiatric Services, Emory University Student Health and Counseling Services , Atlanta , Georgia
org： Department of Psychiatry and Behavioral Sciences , The George Washington University School of Medicine and Health Sciences , Washington , DC
org： Department of Health Policy and Management , Rollins School of Public Health, Emory University , Atlanta , Georgia
org： University Health

org： Department of Health Education, University of California, Santa Barbara
org： Department of Health Education, University of California, Santa Barbara
org： Department of Health Education, University of California, Santa Barbara
org： Mental Health Division, University Health Services, University of Massachusetts , Amherst, Massachusetts, 01003, USA
org： San Diego State University Student Health Center , USA
org： Student Health Center at San Diego State , USA
org： Department of Psychology, Creighton University, Omaha, NE
org： Creighton's peer-education program
org： Creighton's Psychology Department, Medical School
org： Department of Sociomedical Sciences, Mailman School of Public Health , Columbia University , New York
org： Canyon Ranch Institute , Tucson, Arizona
org： Department of Epidemiology and Biostatistics at Stempel School of Public Health, Florida International University, Miami
org： Department of Epidemiology and Biostatistics at Stempel School of Public Health, Florida Inte

org： Department of Public Health Medicine, School of Public Health , University of Bielefeld , Bielefeld, Germany
org： Roger Williams University in Bristol , Rhode Island, USA
org： Center for Gerontology and Health Care Research at Brown University in Providence , Rhode Island, USA
org： National Social Norms Resource Center, Social Science Research Institute, DeKalb, IL
org： National Social Norms Resource Center, Social Science Research Institute, DeKalb, IL
org： National Social Norms Resource Center, Social Science Research Institute, DeKalb, IL
org： Boynton Health Service , 410 Church Street S.E
org： Academic Affairs
org： University of Minnesota , USA
org： Multidisciplinary International Research Training Program, Department of Epidemiology, Harvard University School of Public Health, Boston, Massachusetts
org： Centro de Rehabilitación Club de Leones Cruz del Sur, Punta Arenas, Chile
org： Centro de Rehabilitación Club de Leones Cruz del Sur, Punta Arenas, Chile
org： Centro de Rehabil

org： Research Institute on Addictions , University at Buffalo , Buffalo, New York
org： Research Institute on Addictions , University at Buffalo , Buffalo, New York
org： Research Institute on Addictions , University at Buffalo , Buffalo, New York
org： Research Institute on Addictions , University at Buffalo , Buffalo, New York
org： Department of Community Health , William Paterson College , Wayne, New Jersey, USA
org： University of Rochester , USA
org： School of Public Health
org： School of Public Health
org： School of Public Health
org： United College
org： University Health Service, The Chinese University of Hong Kong , Shatin, Hong Kong
org： School of Public Health
org： School of Public Health
org： School of Public Health
org： School of Public Health
org： Bureau of Study Counsel , Harvard University , 5 Linden Street, Cambridge, Massachusetts, 02138, USA
org： William F. Connell School of Nursing, Boston College, Chestnut Hill, Massachusetts
org： William F. Connell School of Nursing, B

org： University of Florida Student Health Services in Gainesville , USA
org： University of Florida Student Health Services in Gainesville , USA
org： University of Florida Student Health Services in Gainesville , USA
org： Student Health Service, Pima Community College , 2202 W. Anklam Road, Tucson, Arizona, 22030, USA
org： Department of Psychology , Quinnipiac College , USA
org： Student Mental Health Service , University of Connecticut , USA
org： Department of Psychology , University of North Carolina at Charlotte , Charlotte , North Carolina
org： DOGESPORT Group, Seville, Spain
org： CEU Cardenal Herrera University of Elche, Elche, Spain
org： University of Huelva, Huelva, Spain
org： Tierrallana Attendis School, Huelva, Spain
org： Office of Health Promotion and Education for Drug Education , Penn State University , University Park, USA
org： Office of Health Promotion and Education for Sexual Health , USA
org： University Health Services , USA
org： Duke University Medical Center, Durham, N

org： College of Public Health and the Rural Cancer Prevention Center , University of Kentucky , Lexington , Kentucky
org： Department of Behavioral Sciences and Health Education , Rollins School of Public Health, Emory University , Atlanta , Georgia
org： Department of Behavioral Sciences and Health Education , Rollins School of Public Health, Emory University , Atlanta , Georgia
org： Department of Behavioral Sciences and Health Education , Rollins School of Public Health, Emory University , Atlanta , Georgia
org： Department of Psychology , Morehouse College , Atlanta , Georgia
org： Department of Behavioral Sciences and Health Education , Rollins School of Public Health, Emory University , Atlanta , Georgia
org： University Health Services, Harvard University , 75 Mt. Auburn Street, Cambridge, Massachusetts, 02138, USA
org： University of Rochester, School of Medicine and Dentistry , USA
org： University of Maine , Presque Isle, USA
org： Department of Health Education , Southern Illinois Un

org： Department of Psychology , Oklahoma State University , Stillwater , Oklahoma
org： School of Nursing , Indiana University , Indianapolis , Indiana
org： Department of Psychology , Oklahoma State University , Stillwater , Oklahoma
org： Department of Psychology , Oklahoma State University , Stillwater , Oklahoma
org： Department of Psychology , Oklahoma State University , Stillwater , Oklahoma
org： Department of Psychology , Oklahoma State University , Stillwater , Oklahoma
org： Department of Psychology, California State University, Sacramento, Sacramento, California, USA
org： Department of Psychology, California State University, Stanislaus, Turlock, California, USA
org： Department of Internal Medicine, University of California, Davis, Davis, California, USA
org： Department of Psychology, California State University, Sacramento, Sacramento, California, USA
org： Department of Clinical Pediatrics, School of Medicine, University of California, Davis, Davis, California, USA
org： Universit

org： School of Public Health at the University of North Carolina in Chapel Hill , USA
org： University of California in Berkeley , USA
org： Department of Psychiatry and Behavioral Neuroscience , University of Chicago , Chicago , Illinois
org： Department of Public Health, Faculty of Health and Medical Sciences , University of Copenhagen , Copenhagen , Denmark
org： Department of Psychiatry and Behavioral Neuroscience , University of Chicago , Chicago , Illinois
org： Division of Epidemiology and Community Health, School of Public Health , University of Minnesota , Minneapolis , Minnesota
org： Boynton Health Services , University of Minnesota , Minneapolis , Minnesota
org： Boynton Health Services , University of Minnesota , Minneapolis , Minnesota
org： Priority Research Centre in Physical Activity and Nutrition, School of Education, University of Newcastle, Callaghan Campus, Australia
org： Priority Research Centre in Physical Activity and Nutrition, School of Education, University of Newcas

## 5. 检查email字段

**(1) 检查 email字段数据问题**

In [37]:
if 'authors' in data.columns:
    have_authors = data[data['authors'].notnull()]
    tag = 0
    for i in range(len(have_authors)):
        if len(have_authors.iloc[i]['authors']) != 0:
            for n in range(len(have_authors.iloc[i]['authors'])):
                authors = have_authors.iloc[i]['authors'][n]
                if 'email' in authors:
                    emailstr = authors['email']
                    if emailstr :
                        if ('\n' in emailstr) or ('\t' in emailstr) or ('\r' in emailstr):
                            tag = 1
                        if (' ' in emailstr) or ('@' not in emailstr):
                            tag = 1
                        if ('(' in emailstr) or (not emailstr[0].isalpha()) or (not emailstr[-1].isalpha()):
                            if (emailstr[0] == ' ') or (emailstr[-1] == ' '):
                                tag = 1
                    else:
                        tag = 1
    if tag == 0:
        print('正常，未发现email存在问题')
    else:
        print('异常，email字段可能出现问题')
        print('请详细检查以下email是否出现问题：\n')
    
        for i in range(len(have_authors)):
            if len(have_authors.iloc[i]['authors']) != 0:
                for n in range(len(have_authors.iloc[i]['authors'])):
                    authors = have_authors.iloc[i]['authors'][n]
                    if 'email' in authors:
                        emailstr = authors['email']
                        if emailstr :
                            if ('\n' in emailstr) or ('\t' in emailstr) or ('\r' in emailstr):
                                print('email中可能存在转义字符 ntr')
                                print('email：',emailstr)
                                print('url：',have_authors.iloc[i]['url'][0],'\n')
                            if (' ' in emailstr) or ('@' not in emailstr):
                                print('email中可能存在异常')
                                print('email：',emailstr)
                                print('url：',have_authors.iloc[i]['url'][0],'\n')
                            if ('(' in emailstr) or (not emailstr[0].isalnum()) or (not emailstr[-1].isalpha()):
                                if emailstr.strip() == '':
                                    print('email可能为空')
                                    print('email：',emailstr)
                                    print('url：',have_authors.iloc[i]['url'][0],'\n')
                                elif (emailstr[0] == ' ') or (emailstr[-1] == ' '):
                                    print('email两侧可能存在空格')
                                    print('email：',emailstr)
                                    print('url：',have_authors.iloc[i]['url'][0],'\n')
                        else:
                                print('email为空')
                                print('email：',emailstr)
                                print('url：',have_authors.iloc[i]['url'][0],'\n')
else:
    print('异常，可能缺少字段authors，请详细检查数据源')

正常，未发现email存在问题


**(2) 浏览检查 email字段**

In [38]:
if 'authors' in data.columns:
    have_authors = data[data['authors'].notnull()]
    for i in range(len(have_authors)):
        if len(have_authors) != 0 :
            for n in range(len(have_authors.iloc[i]['authors'])):
                author = have_authors.iloc[i]['authors'][n]
                if 'email' in author:
                    emailstr = author['email']
                    print('email：',emailstr)
                    # 下面一行代码用于显示url，有问题时候可取消注释
#                     print('url : ',have_authors.iloc[i]['url'][0],'\n')
else:
    print('异常，可能缺少字段authors，请详细检查数据源')

email： rosenmol@indiana.edu
email： carly.thompson@rockets.utoledo.edu
email： arobilla@mailbox.sc.edu
email： angela.m.heads@uth.tmc.edu
email： Whitney.linsenmeyer@health.slu.edu
email： kmincey@xula.edu
email： kohhye@gmail.com
email： rburesh@kennesaw.edu
email： sara.oswalt@utsa.edu
email： zjackson13@tamu.edu
email： helee@hufs.ac.kr
email： hahmbj@gmail.com;hahm@snu.ac.kr
email： marquitta.webb@sta.uwi.edu
email： dshell2@unl.edu
email： ramon.hinojosa@ucf.edu
email： d.giovenco@rutgers.edu
email： angela.robertson@ssrc.msstate.edu
email： jennifer.McNeely@nyumc.org
email： Jess.Shatkin@nyumc.org
email： kimm@nsuok.edu
email： abraitma@odu.edu
email： Tiff.marcan@gmail.com
email： sharyn.potter@unh.edu
email： kwagoner@wakehealth.edu
email： ts11jy@brocku.ca
email： jsu2@vcu.edu
email： sezerkisa@hotmail.com
email： jvn11@georgetown.edu
email： ahconley@vcu.edu
email： abamstadter@vcu.edu
email： melody.noland@uky.edu
email： kristie.thomas@simmons.edu
email： brian.borsari@va.gov;brian.borsari@ucsf.edu
email：

email： Hunter.Spotts@duke.edu
email： wjl88816@163.com
email： jbantjes@sun.ac.za
email： Deana.Davalos@ColoState.Edu
email： jlbourdon@vcu.edu
email： k.hayes@mail.utoronto.ca
email： scott.butler@gcsu.edu
email： lmsmiller@ucdavis.edu
email： gerald@bu.edu
email： gozde0007@gmail.com
email： jenirwin@uwo.ca
email： matthew.keough@umanitoba.ca
email： khopkins@prc.utexas.edu
email： megan.moreno@seattlehchildrens.org
email： blyerkb@jmu.edu
email： cgipson@uttyler.edu
email： Ron.Plotnikoff@newcastle.edu.au
email： kaestle@vt.edu
email： lacharya@purdue.edu
email： Kate_Carey@brown.edu
email： seg10e@my.fsu.edu
email： parks@ria.buffalo.edu
email： gerrit.vanschalkwyk@yale.edu
email： metzgeri@musc.edu
email： eastmanmuellerh@health.missouri.edu
email： mary.hayney@wisc.edu
email： gjone046@uottawa.ca
email： jablayne@buffalo.edu
email： alexis.blavos@cortland.edu
email： Tavis.glassman@utoledo.edu
email： jablayne@buffalo.edu
email： ali_crandall@byu.edu
email： hirsch@etsu.edu
email： jmcderm@emory.edu
email： sdyou

# =============
# 五. reference、citation字段查询
# =============

## 1. 检查reference字段

**(1) 检查 reference字段数据格式（reference-List[Html String]）**

In [39]:
if 'reference' in data.columns:
    have_reference = data[data['reference'].notnull()]
    reference_type = []
    reference_type_warn = 0

    for i in range(len(have_reference)):
        reference_type.append(str(isinstance(have_reference.iloc[i]['reference'],list)))
        if not isinstance(have_reference.iloc[i]['reference'],list):
            reference_type_warn = reference_type_warn + 1
        else:
            if (have_reference.iloc[i]['reference'][0] != '<') or (have_reference.iloc[i]['reference'][-1] != '>') :
                reference_type_warn = reference_type_warn + 1
    if 'False' not in reference_type:
        print('正常，reference数据类型正确')
    else:
        print('异常，reference数据类型可能存在错误')
        print('【reference数据类型可能存在错误的数据量 / 存在reference的数据总量】  <==>  【',reference_type_warn,'/',len(have_reference),'】\n')
        for i in range(len(have_reference)):
            if not isinstance(have_reference.iloc[i]['reference'],str):
                print('出现问题的reference数据类型：',type(have_reference.iloc[i]['reference']))
                print('url：',have_reference.iloc[i]['url'][0],'\n')
            else:
                if (have_reference.iloc[i]['reference'][0] != '<') or (have_reference.iloc[i]['reference'][-1] != '>') :
                    print('reference可能非Html String类型：')
                    print('url：',have_reference.iloc[i]['url'][0],'\n')
else:
    print('异常，可能缺少字段reference，请详细检查数据源')

正常，reference数据类型正确


**(2) 检查 reference字段数据缺失**

In [40]:
if 'reference' in data.columns:
    no_reference = data[data['reference'].isnull()]
    if len(no_reference) == 0:
        print('正常，未发现reference字段缺失情况')
    else:
        print('异常，可能存在reference缺失现象')
        print('【reference数据类型可能缺失的数据量 / 数据总量】  <==>  【',len(no_reference),'/',len(data),'】')
        print('请从以下随机抽样检查，查看是否存在抓取遗漏：\n')
        for i in range(len(no_reference)):
            print('url：',no_reference.iloc[i]['url'][0])
else:
    print('异常，可能缺少字段reference，请详细检查数据源')

异常，可能存在reference缺失现象
【reference数据类型可能缺失的数据量 / 数据总量】  <==>  【 423 / 2484 】
请从以下随机抽样检查，查看是否存在抓取遗漏：

url： https://www.tandfonline.com/doi/abs/10.1080/01644300.1981.10393046
url： https://www.tandfonline.com/doi/abs/10.1080/01644300.1981.10393028
url： https://www.tandfonline.com/doi/abs/10.1080/01644300.1981.10393064
url： https://www.tandfonline.com/doi/abs/10.1080/01644300.1980.10392950
url： https://www.tandfonline.com/doi/abs/10.1080/07448481.1987.9939648
url： https://www.tandfonline.com/doi/abs/10.1080/07448481.1994.9939081
url： https://www.tandfonline.com/doi/abs/10.1080/07448481.1981.9938888
url： https://www.tandfonline.com/doi/abs/10.1080/07448480109596018
url： https://www.tandfonline.com/doi/abs/10.1080/07448481.1991.9936234
url： https://www.tandfonline.com/doi/full/10.1080/07448481.2014.992723
url： https://www.tandfonline.com/doi/abs/10.1080/07448489909595671
url： https://www.tandfonline.com/doi/abs/10.1080/07448481.1985.9938930
url： https://www.tandfonline.com/doi/abs/10.1080/01644

## 2. 检查citation字段

**(1) 检查 citation字段数据格式（citation-List[Html String]）**

In [41]:
if 'citation' in data.columns:
    have_citation = data[data['citation'].notnull()]
    citation_type = []
    citation_type_warn = 0

    for i in range(len(have_citation)):
        citation_type.append(str(isinstance(have_citation.iloc[i]['citation'],list)))
        if not isinstance(have_citation.iloc[i]['citation'],list):
            citation_type_warn = citation_type_warn + 1
        else:
            if (have_citation.iloc[i]['citation'][0] != '<') or (have_citation.iloc[i]['citation'][-1] != '>') :
                citation_type_warn = citation_type_warn + 1
    if 'False' not in citation_type:
        print('正常，citation数据类型正确')
    else:
        print('异常，citation数据类型可能存在错误')
        print('【citation数据类型可能存在错误的数据量 / 存在citation的数据总量】  <==>  【',citation_type_warn,'/',len(have_citation),'】\n')
        for i in range(len(have_citation)):
            if not isinstance(have_citation.iloc[i]['citation'],list):
                print('出现问题的citation数据类型：',type(have_citation.iloc[i]['citation']))
                print('url：',have_citation.iloc[i]['url'][0],'\n')
            else:
                if (have_citation.iloc[i]['citation'][0] != '<') or (have_citation.iloc[i]['citation'][-1] != '>') :
                    print('citation可能非Html String类型：')
                    print('url：',have_citation.iloc[i]['url'][0],'\n')
else:
    print('异常，可能缺少字段citation，请详细检查数据源')

正常，citation数据类型正确


**(2) 检查 citation字段数据缺失**

In [42]:
if 'citation' in data.columns:
    no_citation = data[data['citation'].isnull()]
    if len(no_citation) == 0:
        print('正常，未发现citation字段缺失情况')
    else:
        print('异常，可能存在citation缺失现象')
        print('【citation数据类型可能缺失的数据量 / 数据总量】  <==>  【',len(no_citation),'/',len(data),'】')
        print('请从以下随机抽样检查，查看是否存在抓取遗漏：\n')
        for i in range(len(no_citation)):
            print('url：',no_citation.iloc[i]['url'][0])
else:
    print('异常，可能缺少字段citation，请详细检查数据源')

异常，可能存在citation缺失现象
【citation数据类型可能缺失的数据量 / 数据总量】  <==>  【 610 / 2484 】
请从以下随机抽样检查，查看是否存在抓取遗漏：

url： https://www.tandfonline.com/doi/abs/10.1080/01644300.1981.10393046
url： https://www.tandfonline.com/doi/abs/10.1080/01644300.1981.10393017
url： https://www.tandfonline.com/doi/abs/10.1080/01644300.1981.10393028
url： https://www.tandfonline.com/doi/abs/10.1080/01644300.1979.10392875
url： https://www.tandfonline.com/doi/abs/10.1080/01644300.1981.10393064
url： https://www.tandfonline.com/doi/full/10.1080/07448481.2018.1472096
url： https://www.tandfonline.com/doi/full/10.1080/07448481.2017.1400975
url： https://www.tandfonline.com/doi/full/10.1080/07448481.2018.1431917
url： https://www.tandfonline.com/doi/abs/10.1080/01644300.1980.10392950
url： https://www.tandfonline.com/doi/abs/10.1080/07448481.1987.9939648
url： https://www.tandfonline.com/doi/abs/10.1080/07448481.1983.9939557
url： https://www.tandfonline.com/doi/abs/10.1080/07448481.1981.9938888
url： https://www.tandfonline.com/doi/full/1

# =============
# 五. src、sid、lang、venue、ts字段检查
# =============

## 1. 检查src字段

**(1) 检查 src字段数据类型**

In [43]:
if 'src' in data.columns:
    have_src = data[data['src'].notnull()]
    src_type = []
    for i in range(len(have_src)):
        src_type.append(str(isinstance(have_src.iloc[i]['src'],str)))
    if 'False' not in src_type:
        print('正常，src数据类型正确')
    else:
        print('异常，src数据类型可能存在错误\n')
        for i in range(len(have_src)):
            src = have_src.iloc[i]['src']
            if not isinstance(src,str):
                print('src的数据类型为：',type(src))
                print('url：',have_src.iloc[i]['url'][0])
else:
    print('异常，可能缺少必填字段src')

正常，src数据类型正确


**(2) 检查 src字段数据问题**

In [44]:
if 'src' in data.columns:
    have_src = data[data['src'].notnull()]
    src = have_src.iloc[0]['src']
    print('请详细检查，src字段应为 数据源名称')
    print('src：',src)
else:
    print('异常，可能缺少必填字段src')

请详细检查，src字段应为 数据源名称
src： Taylor & Francis Group


## 2. 检查sid字段

**(1) 检查 sid字段数据重复**

In [45]:
sidlist = []
for sid in data['sid']:
    sidlist.append(sid)

sid_list = list(set(sidlist))

if len(sid_list) == len(sidlist):
    print('正常，未发现重复数据！')
else:
    print('异常，可能存在重复数据')
    print('【重复数据量 / 数据总量】  <==>  【',len(sidlist)-len(sid_list),'/',len(data),'】\n')

    #展现重复元素和重复次数
    repeation_tag = dict(Counter(sidlist))
    for key,value in repeation_tag.items():
        if value>1:
            print('sid：',key,'\t出现次数：',value)

正常，未发现重复数据！


**(2) 浏览检查 sid字段数据**

In [46]:
if 'sid' in data.columns:
    have_sid = data[data['sid'].notnull()]
    print('请详细检查，sid字段应为 所属数据源的论文唯一标识符')
    for i in range(len(have_sid)):
        print('sid：',have_sid.iloc[i]['sid'])
        print('url：',have_sid.iloc[i]['url'][0],'\n')
else:
    print('异常，可能缺少必填字段sid')

请详细检查，sid字段应为 所属数据源的论文唯一标识符
sid： 10.1080/07448481.2012.750607
url： https://www.tandfonline.com/doi/full/10.1080/07448481.2012.750607 

sid： 10.1080/07448481.2018.1462817
url： https://www.tandfonline.com/doi/full/10.1080/07448481.2018.1462817 

sid： 10.1080/01644300.1981.10393046
url： https://www.tandfonline.com/doi/abs/10.1080/01644300.1981.10393046 

sid： 10.1080/07448481.2018.1500470
url： https://www.tandfonline.com/doi/full/10.1080/07448481.2018.1500470 

sid： 10.1080/01644300.1981.10393017
url： https://www.tandfonline.com/doi/abs/10.1080/01644300.1981.10393017 

sid： 10.3200/JACH.57.2.211-220
url： https://www.tandfonline.com/doi/abs/10.3200/JACH.57.2.211-220 

sid： 10.108007448481.2010.502196
url： https://www.tandfonline.com/doi/full/10.1080/07448481.2010.502196 

sid： 10.108007448481.2010.519014
url： https://www.tandfonline.com/doi/full/10.1080/07448481.2010.519014 

sid： 10.1080/07448481.1995.9939107
url： https://www.tandfonline.com/doi/abs/10.1080/07448481.1995.9939107 

sid： 10

url： https://www.tandfonline.com/doi/abs/10.1080/01644300.1981.10393050 

sid： 10.1080/07448481.2016.1229320
url： https://www.tandfonline.com/doi/full/10.1080/07448481.2016.1229320 

sid： 10.1080/07448481.1993.9936338
url： https://www.tandfonline.com/doi/abs/10.1080/07448481.1993.9936338 

sid： 10.1080/07448481.2015.1015031
url： https://www.tandfonline.com/doi/full/10.1080/07448481.2015.1015031 

sid： 10.1080/01644300.1979.10392891
url： https://www.tandfonline.com/doi/abs/10.1080/01644300.1979.10392891 

sid： 10.1080/07448481.2015.1085057
url： https://www.tandfonline.com/doi/full/10.1080/07448481.2015.1085057 

sid： 10.1080/07448481.1993.9936350
url： https://www.tandfonline.com/doi/abs/10.1080/07448481.1993.9936350 

sid： 10.1080/07448481.1983.10392796
url： https://www.tandfonline.com/doi/abs/10.1080/07448481.1983.10392796 

sid： 10.3200/JACH.55.6.341-350
url： https://www.tandfonline.com/doi/abs/10.3200/JACH.55.6.341-350 

sid： 10.1080/01644300.1979.10392906
url： https://www.tandfonlin

url： https://www.tandfonline.com/doi/abs/10.1080/01644300.1980.10392989 

sid： 10.1080/07448481.2010.528097
url： https://www.tandfonline.com/doi/full/10.1080/07448481.2010.528097 

sid： 10.3200/JACH.57.5.561-564
url： https://www.tandfonline.com/doi/abs/10.3200/JACH.57.5.561-564 

sid： 10.3200/JACH.55.4.219-225
url： https://www.tandfonline.com/doi/abs/10.3200/JACH.55.4.219-225 

sid： 10.1080/01644300.1981.10393012
url： https://www.tandfonline.com/doi/abs/10.1080/01644300.1981.10393012 

sid： 10.3200/JACH.54.1.45-49
url： https://www.tandfonline.com/doi/abs/10.3200/JACH.54.1.45-49 

sid： 10.1080/07448481.2014.963106
url： https://www.tandfonline.com/doi/full/10.1080/07448481.2014.963106 

sid： 10.1080/07448481.1982.9936130
url： https://www.tandfonline.com/doi/abs/10.1080/07448481.1982.9936130 

sid： 10.1080/07448481.2016.1179196
url： https://www.tandfonline.com/doi/full/10.1080/07448481.2016.1179196 

sid： 10.1080/07448481.2011.582208
url： https://www.tandfonline.com/doi/full/10.1080/07448

url： https://www.tandfonline.com/doi/abs/10.3200/JACH.56.44.359-368 

sid： 10.1080/07448489809595626
url： https://www.tandfonline.com/doi/abs/10.1080/07448489809595626 

sid： 10.1080/07448481.1982.9938897
url： https://www.tandfonline.com/doi/abs/10.1080/07448481.1982.9938897 

sid： 10.1080/07448481.1984.9939576
url： https://www.tandfonline.com/doi/abs/10.1080/07448481.1984.9939576 

sid： 10.1080/07448481.2011.630702
url： https://www.tandfonline.com/doi/full/10.1080/07448481.2011.630702 

sid： 10.1080/07448480109596314
url： https://www.tandfonline.com/doi/abs/10.1080/07448480109596314 

sid： 10.1080/07448481.1982.9938900
url： https://www.tandfonline.com/doi/abs/10.1080/07448481.1982.9938900 

sid： 10.1080/07448481.1992.9936294
url： https://www.tandfonline.com/doi/abs/10.1080/07448481.1992.9936294 

sid： 10.1080/01644300.1979.10392860
url： https://www.tandfonline.com/doi/abs/10.1080/01644300.1979.10392860 

sid： 10.1080/01644300.1981.10393015
url： https://www.tandfonline.com/doi/abs/10.1

sid： 10.1080/07448480903221327
url： https://www.tandfonline.com/doi/full/10.1080/07448480903221327 

sid： 10.1080/07448481.2016.1233557
url： https://www.tandfonline.com/doi/full/10.1080/07448481.2016.1233557 

sid： 10.1080/07448481.1983.9939571
url： https://www.tandfonline.com/doi/abs/10.1080/07448481.1983.9939571 

sid： 10.1080/07448481.1985.9939627
url： https://www.tandfonline.com/doi/abs/10.1080/07448481.1985.9939627 

sid： 10.3200/JACH.55.4.195-206
url： https://www.tandfonline.com/doi/abs/10.3200/JACH.55.4.195-206 

sid： 10.1080/07448481.2013.865626
url： https://www.tandfonline.com/doi/full/10.1080/07448481.2013.865626 

sid： 10.1080/07448481.2018.1440583
url： https://www.tandfonline.com/doi/full/10.1080/07448481.2018.1440583 

sid： 10.1080/07448481.1984.10392812
url： https://www.tandfonline.com/doi/abs/10.1080/07448481.1984.10392812 

sid： 10.1080/07448481.1983.9936133
url： https://www.tandfonline.com/doi/abs/10.1080/07448481.1983.9936133 

sid： 10.3200/JACH.58.1.91-92
url： https:

sid： 10.1080/07448481.2012.716981
url： https://www.tandfonline.com/doi/full/10.1080/07448481.2012.716981 

sid： 10.1080/07448480903501764
url： https://www.tandfonline.com/doi/full/10.1080/07448480903501764 

sid： 10.1080/07448481.1991.9936238
url： https://www.tandfonline.com/doi/abs/10.1080/07448481.1991.9936238 

sid： 10.1080/07448480009596290
url： https://www.tandfonline.com/doi/abs/10.1080/07448480009596290 

sid： 10.1080/07448481.1996.9937521
url： https://www.tandfonline.com/doi/abs/10.1080/07448481.1996.9937521 

sid： 10.3200/JACH.57.6.611-618
url： https://www.tandfonline.com/doi/abs/10.3200/JACH.57.6.611-618 

sid： 10.1080/07448481.1995.9937504
url： https://www.tandfonline.com/doi/abs/10.1080/07448481.1995.9937504 

sid： 10.1080/07448481.1986.9938957
url： https://www.tandfonline.com/doi/abs/10.1080/07448481.1986.9938957 

sid： 10.1080/07448481.2010.519013
url： https://www.tandfonline.com/doi/full/10.1080/07448481.2010.519013 

sid： 10.1080/01644300.1980.10392962
url： https://www.

url： https://www.tandfonline.com/doi/full/10.1080/07448481.2016.1210612 

sid： 10.1080/07448481.1982.9938907
url： https://www.tandfonline.com/doi/abs/10.1080/07448481.1982.9938907 

sid： 10.1080/07448481.1991.9936230
url： https://www.tandfonline.com/doi/abs/10.1080/07448481.1991.9936230 

sid： 10.1080/07448481.1982.9938923
url： https://www.tandfonline.com/doi/abs/10.1080/07448481.1982.9938923 

sid： 10.1080/01644300.1981.10393010
url： https://www.tandfonline.com/doi/abs/10.1080/01644300.1981.10393010 

sid： 10.1080/07448481.1994.9938445
url： https://www.tandfonline.com/doi/abs/10.1080/07448481.1994.9938445 

sid： 10.1080/07448489909595645
url： https://www.tandfonline.com/doi/abs/10.1080/07448489909595645 

sid： 10.1080/07448481003621718
url： https://www.tandfonline.com/doi/full/10.1080/07448481003621718 

sid： 10.1080/07448481.2017.1341901
url： https://www.tandfonline.com/doi/full/10.1080/07448481.2017.1341901 

sid： 10.1080/07448480209603452
url： https://www.tandfonline.com/doi/abs/10

sid： 10.1080/07448481.1989.9937495
url： https://www.tandfonline.com/doi/abs/10.1080/07448481.1989.9937495 

sid： 10.1080/07448481.1984.9936170
url： https://www.tandfonline.com/doi/abs/10.1080/07448481.1984.9936170 

sid： 10.1080/07448481.1984.10392806
url： https://www.tandfonline.com/doi/abs/10.1080/07448481.1984.10392806 

sid： 10.3200/JACH.56.44.391-394
url： https://www.tandfonline.com/doi/abs/10.3200/JACH.56.44.391-394 

sid： 10.1080/07448481.1984.10392811
url： https://www.tandfonline.com/doi/abs/10.1080/07448481.1984.10392811 

sid： 10.1080/01644300.1979.10392893
url： https://www.tandfonline.com/doi/abs/10.1080/01644300.1979.10392893 

sid： 10.1080/01644300.1979.10392925
url： https://www.tandfonline.com/doi/abs/10.1080/01644300.1979.10392925 

sid： 10.1080/07448481.1990.9936220
url： https://www.tandfonline.com/doi/abs/10.1080/07448481.1990.9936220 

sid： 10.1080/07448481.2014.891593
url： https://www.tandfonline.com/doi/full/10.1080/07448481.2014.891593 

sid： 10.3200/JACH.54.2.69-7

url： https://www.tandfonline.com/doi/abs/10.1080/07448481.1991.9936237 

sid： 10.1080/07448481.1983.9939575
url： https://www.tandfonline.com/doi/abs/10.1080/07448481.1983.9939575 

sid： 10.1080/07448481.1994.9938443
url： https://www.tandfonline.com/doi/abs/10.1080/07448481.1994.9938443 

sid： 10.1080/07448481.2017.1312409
url： https://www.tandfonline.com/doi/full/10.1080/07448481.2017.1312409 

sid： 10.1080/07448481.2018.1464335
url： https://www.tandfonline.com/doi/full/10.1080/07448481.2018.1464335 

sid： 10.1080/07448481.1988.9939046
url： https://www.tandfonline.com/doi/abs/10.1080/07448481.1988.9939046 

sid： 10.3200/JACH.55.3.126-260
url： https://www.tandfonline.com/doi/abs/10.3200/JACH.55.3.126-260 

sid： 10.1080/07448481.1984.9936166
url： https://www.tandfonline.com/doi/abs/10.1080/07448481.1984.9936166 

sid： 10.1080/07448481.2010.513407
url： https://www.tandfonline.com/doi/full/10.1080/07448481.2010.513407 

sid： 10.1080/07448481.1997.9936899
url： https://www.tandfonline.com/do

## 3. 检查lang字段

**(1) 检查 lang字段数据类型**

In [47]:
if 'lang' in data.columns:
    have_lang = data[data['lang'].notnull()]
    lang_type = []
    for i in range(len(have_lang)):
        lang_type.append(str(isinstance(have_lang.iloc[i]['lang'],str)))
    if 'False' not in lang_type:
        print('正常，lang数据类型正确')
    else:
        print('异常，lang数据类型可能存在错误\n')
        for i in range(len(have_lang)):
            lang = have_lang.iloc[i]['lang']
            if not isinstance(lang,str):
                print('lang的数据类型为：',type(lang))
                print('url：',have_lang.iloc[i]['url'][0])
else:
    print('异常，可能缺少必填字段lang')

正常，lang数据类型正确


**(2) 检查 lang字段数据问题**

In [48]:
if 'lang' in data.columns:
    have_lang = data[data['lang'].notnull()]
    lang = have_lang.iloc[i]['lang']
    if (lang != 'en') and (lang != "ch") and (lang != 'fr') and (lang != 'de') and (lang != 'it'):
        # 英语；汉语；法语；德语；意大利语
        print('异常，发现数lang字段可能出现问题，请详细检查\n')
    else:
        print('正常，未发现lang字段存在数据问题\n')
    print('lang：',lang)
else:
    print('异常，可能缺少必填字段lang')

正常，未发现lang字段存在数据问题

lang： en


## 4. 检查venue字段

**(1) 检查 venue字段数据类型**

In [49]:
if 'venue' in data.columns:
    have_venue = data[data['venue'].notnull()]
    venue_type = []
    venue_type.append(str(isinstance(have_venue.iloc[0]['venue'],dict)))
    venue_type.append(str(isinstance(have_venue.iloc[0]['venue']['name'],str)))
    venue_type.append(str(isinstance(have_venue.iloc[0]['venue']['type'],int)))
    venue_type.append(str(isinstance(have_venue.iloc[0]['venue']['sid'],str)))
    if 'False' not in venue_type:
        print('正常，venue数据类型正确')
    else:
        print('异常，venue数据类型可能存在错误\n')
        venue = have_venue.iloc[0]['venue']
        venue_name = have_venue.iloc[0]['venue']['name']
        venue_type = have_venue.iloc[0]['venue']['type']
        venue_sid = have_venue.iloc[0]['venue']['sid']
        if (not isinstance(venue,dict)):
            print('venue的数据类型应为：dict')
            print('venue的数据类型为：',type(venue))
        if (not isinstance(venue_name,str)):
            print('venue_name的数据类型应为：String')
            print('venue_name的数据类型为：',type(venue_name),'\n')
        if (not isinstance(venue_type,int)):
            print('venue_type的数据类型应为：Int')
            print('venue_type的数据类型为：',type(venue_type),'\n')
        if (not isinstance(venue_sid,str)):
            print('venue_sid的数据类型应为：String')
            print('venue_sid的数据类型为：',type(venue_sid),'\n')
else:
    print('异常，可能缺少必填字段venue')

正常，venue数据类型正确


**(2) 检查 venue字段数据重复**

In [50]:
venue_name = []
venue_type = []
venue_sid = []
for each in data['venue']:
    venue_name.append(each['name'])
    venue_type.append(each['type'])
    venue_sid.append(each['sid'])
    
if len(set(venue_name)) == 1:
    print('正常，未发现venue_name字段重复')
else:
    print('venue中的name不止一个，请重新检查：',set(venue_name))
if len(set(venue_type)) == 1:
    print('正常，未发现venue_type字段重复')
else:
    print('venue中的type不止一个，请重新检查：',set(venue_sid))
if len(set(venue_sid)) == 1:
    print('正常，未发现venue_sid字段重复')
else:
    print('venue中的sid不止一个，请重新检查：',set(venue_sid))

正常，未发现venue_name字段重复
正常，未发现venue_type字段重复
正常，未发现venue_sid字段重复


**(3) 检查 venue字段数据问题**

In [51]:
if 'venue' in data.columns:
    have_venue = data[data['venue'].notnull()]
    venue = have_venue.iloc[0]['venue']
    venue_name = have_venue.iloc[0]['venue']['name']
    venue_type = have_venue.iloc[0]['venue']['type']
    venue_sid = have_venue.iloc[0]['venue']['sid']
    print('请详细检查，以下venue字段是否符合规定\n')
    print('venue_name应为：论文中的期刊会议书籍名称')
    print('venue_name为：',venue_name,'\n')
    print('venue_type应为：在venue表中定义的类别')
    print('venue_type为：',venue_type,'\n')
    print('venue_sid应为：在数据源中的Source id')
    print('venue_sid为：',venue_sid,'\n')
else:
    print('异常，可能缺少必填字段venue')

请详细检查，以下venue字段是否符合规定

venue_name应为：论文中的期刊会议书籍名称
venue_name为： Journal of American College Health 

venue_type应为：在venue表中定义的类别
venue_type为： 1 

venue_sid应为：在数据源中的Source id
venue_sid为： vach20 



## 5. 检查ts字段

**(1) 检查 ts字段数据类型**

In [52]:
if 'ts' in data.columns:
    if '$date' in data.iloc[0]['ts'] :
        ts = data.iloc[0]['ts']['$date']
        if 'T' in ts:
            ts = ts.split('.')[0].replace('T',' ')
            try:
                time.strptime(ts,"%Y-%m-%d %H:%M:%S")
                tag = 0
            except:
                tag = 1
        else:
            tag = 1
        if tag == 0:
            print('正常，字段ts的数据格式可能正确')
        else:
            print('异常，字段ts的数据格式可能出现问题')
        print(data.iloc[0]['ts']['$date'])
    else:
        print('异常，字段ts的数据格式可能出现问题')
else:
    print('异常，可能缺少必填字段ts')

正常，字段ts的数据格式可能正确
2020-01-09T18:51:52.000+0000


# =============
# 六. issn、isbn、doi、pdf_src、paper_type、editor字段检查
# =============

## 1. 检查issn字段

**(1) 检查 issn字段数据格式（issn-Sting）**

In [53]:
if 'issn' in data.columns:
    issn = data['issn'][0]
    tag = 0
    if not isinstance(issn,str):
        tag = 1
    if tag == 1:
        print('异常，issn数据类型可能存在错误')
        print('数据集中issn的数据类型为：',type(issn))
    else:
        print('正常，未发现issn数据类型错误')
else:
    print('异常，可能缺少字段issn，请详细检查数据源')

正常，未发现issn数据类型错误


**(2) 检查 issn字段数据问题**

In [54]:
if 'issn' in data.columns:
    issn = list(set(data['issn']))
    tag = 0
    if (len(issn) != 1) or (len(issn[0]) != 9) or (not data['issn'][0][0].isdigit()) or (not data['issn'][0][-1].isdigit()):
        tag = 1
    if tag == 1:
        print('异常，issn可能出现问题')
        if (len(issn) != 1):
            print('issn不唯一，请详细检查数据源')
        if (len(issn[0]) != 9):
            print('issn可能出现问题，请详细检查')
        if (not data['issn'][0][0].isdigit()) or (not data['issn'][0][-1].isdigit()):
            print('issn两端不为数字，请详细检查')
    else:
        print('正常，未发现issn存在问题')
    for i in range(len(issn)):
        print('issn：',issn[i])
else:
    print('异常，可能缺少字段issn，请详细检查数据源')

正常，未发现issn存在问题
issn： 0744-8481


## 2. 检查isbn字段

**(1) 检查 isbn字段数据格式（issn-Sting）**

In [55]:
if 'isbn' in data.columns:
    isbn = data['isbn'][0]
    tag = 0
    if not isinstance(isbn,str):
        tag = 1
    if tag == 1:
        print('异常，isbn数据类型可能存在错误')
        print('数据集中isbn的数据类型为：',type(isbn))
    else:
        print('正常，未发现isbn数据类型错误')
else:
    print('异常，可能缺少字段isbn，请详细检查数据源')

异常，可能缺少字段isbn，请详细检查数据源


**(2) 检查 isbn字段数据问题**

In [56]:
if 'isbn' in data.columns:
    isbn = list(set(data['isbn']))
    tag = 0
    if (not data['isbn'][0][0].isdigit()) or (not data['isbn'][0][-1].isdigit()):
        tag = 1
    if tag == 1:
        print('异常，isbn可能出现问题')
        if (not data['isbn'][0][0].isdigit()) or (not data['isbn'][0][-1].isdigit()):
            print('isbn两端不为数字，请详细检查')
    else:
        print('正常，未发现isbn存在问题\n')
    for i in range(len(isbn)):
        print('isbn：',isbn[i])
else:
    print('异常，可能缺少字段isbn，请详细检查数据源')

异常，可能缺少字段isbn，请详细检查数据源


## 3. 检查doi字段

**(1) 检查 doi字段数据类型**

In [57]:
if 'doi' in data.columns:
    have_doi = data[data['doi'].notnull()]
    doi_type = []
    for i in range(len(have_doi)):
        doi_type.append(str(isinstance(have_doi.iloc[i]['doi'],str)))
    if 'False' not in doi_type:
        print('正常，doi数据类型正确')
    else:
        print('异常，doi数据类型可能存在错误\n')
        for i in range(len(have_doi)):
            doi = have_doi.iloc[i]['doi']
            if not isinstance(doi,str):
                print('doi的数据类型为：',type(doi))
                print('url：',have_doi.iloc[i]['url'][0])
else:
    print('异常，可能缺少字段doi，请详细检查数据源')

正常，doi数据类型正确


**(2) 检查 doi字段缺失问题**

In [58]:
if 'doi' in data.columns:
    no_doi = data[data['doi'].isnull()]
    if len(no_doi) == 0:
        print('正常，未发现doi缺失现象')
    else:
        print('异常，可能存在doi缺失现象')
        print('【doi数据类型可能缺失的数据量 / 数据总量】  <==>  【',len(no_doi),'/',len(data),'】')
        print('请从以下随机抽样检查，查看是否存在抓取遗漏：\n')
        for i in range(len(no_doi)):
            print('url：',no_doi.iloc[i]['url'][0])
else:
    print('异常，可能缺少字段doi，请详细检查数据源')

正常，未发现doi缺失现象


**(3) 检查 doi字段重复问题**

In [59]:
if 'doi' in data.columns:
    doi_notnull = len(data[data['doi'].notnull()])
    set_doi = len(set(data[data['doi'].notnull()]['doi']))
    if doi_notnull == set_doi:
        print('正常，未发现doi中存在重复数据')
    else:
        print('异常，doi存在 '+ str(doi_notnull-set_doi) +' 条重复数据\n')
        
    repeation_tag = dict(Counter(data['doi']))
    for key,value in repeation_tag.items():
        if value>1:
            if str(key) != 'nan':
                print('doi：',key,'\t出现次数：',value)

正常，未发现doi中存在重复数据


**(4) 检查 doi字段数据问题**

In [60]:
if 'doi' in data.columns:
    have_doi = data[data['doi'].notnull()]
    tag = 0
    for i in range(len(have_doi)):
        doi = have_doi.iloc[i]['doi'].strip()
        if len(doi) == 1:
            if doi[0] == 'nan':
                continue
            else:
                if tag == 0:
                    tag = 1
                    print('异常，doi字段可能出现问题')
                    print('请详细检查以下doi是否出现问题：\n')
                print('doi：',have_doi.iloc[i]['doi'])
                print('url：',have_doi.iloc[i]['url'][0],'\n')
        if (not have_doi.iloc[i]['doi'][0].isalnum()) or (not have_doi.iloc[i]['doi'][-1].isalnum()):
            if tag == 0:
                tag = 1
                print('异常，doi字段可能出现问题')
                print('请详细检查以下doi是否出现问题：\n')
            print('doi：',have_doi.iloc[i]['doi'])
            print('url：',have_doi.iloc[i]['url'][0],'\n')
    if tag == 0:
        print('正常，未发现doi存在问题')
else:
    print('异常，可能缺少字段doi，请详细检查数据源')

正常，未发现doi存在问题


## 4. 检查pdf_src字段

**(1) 检查 pdf_src字段数据类型**

In [61]:
if 'pdf_src' in data.columns:
    have_pdf_src = data[data['pdf_src'].notnull()]
    pdf_src_type = []
    for i in range(len(have_pdf_src)):
        pdf_src_type.append(str(isinstance(have_pdf_src.iloc[i]['pdf_src'],list)))
    if 'False' not in pdf_src_type:
        print('正常，pdf_src数据类型正确')
    else:
        print('异常，pdf_src数据类型可能存在错误\n')
        for i in range(len(have_pdf_src)):
            pdf_src = have_pdf_src.iloc[i]['pdf_src']
            if not isinstance(pdf_src,list):
                print('pdf_src的数据类型为：',type(pdf_src))
                print('url：',have_pdf_src.iloc[i]['url'][0])
else:
    print('异常，可能缺少字段pdf_src，请详细检查数据源')

正常，pdf_src数据类型正确


**(2) 检查 pdf_src字段数据缺失**

In [62]:
if 'pdf_src' in data.columns:
    no_pdf_src = data[data['pdf_src'].isnull()]
    if len(no_pdf_src) == 0:
        print('正常，未发现pdf_src缺失现象')
    else:
        print('异常，可能存在pdf_src缺失现象')
        print('【pdf_src数据类型可能缺失的数据量 / 数据总量】  <==>  【',len(no_pdf_src),'/',len(data),'】')
        print('请从以下随机抽样检查，查看是否存在抓取遗漏：\n')
        for i in range(len(no_pdf_src)):
            print('url：',no_pdf_src.iloc[i]['url'][0])
else:
    print('异常，可能缺少字段pdf_src，请详细检查数据源')

正常，未发现pdf_src缺失现象


## 5. 检查paper_type字段

**(1) 检查 paper_type字段数据类型**

In [63]:
if 'paper_type' in data.columns:
    have_paper_type = data[data['paper_type'].notnull()]
    paper_type_type = []
    for i in range(len(have_paper_type)):
        paper_type_type.append(str(isinstance(have_paper_type.iloc[i]['paper_type'],str)))
    if 'False' not in paper_type_type:
        print('正常，paper_type数据类型正确')
    else:
        print('异常，paper_type数据类型可能存在错误\n')
        for i in range(len(have_paper_type)):
            paper_type = have_paper_type.iloc[i]['paper_type']
            if not isinstance(paper_type,str):
                print('paper_type的数据类型为：',type(paper_type))
                print('url：',have_paper_type.iloc[i]['url'][0])
else:
    print('异常，可能缺少字段paper_type，请详细检查数据源')

正常，paper_type数据类型正确


**(2) 检查 paper_type字段数据缺失**

In [64]:
if 'paper_type' in data.columns:
    no_paper_type = data[data['paper_type'].isnull()]
    if len(no_paper_type) == 0:
        print('正常，未发现paper_type缺失现象')
    else:
        print('异常，可能存在paper_type缺失现象')
        print('【paper_type数据类型可能缺失的数据量 / 数据总量】  <==>  【',len(no_paper_type),'/',len(data),'】')
        print('请从以下随机抽样检查，查看是否存在抓取遗漏：\n')
        for i in range(len(no_paper_type)):
            print('url：',no_paper_type.iloc[i]['url'][0])
else:
    print('异常，可能缺少字段paper_type，请详细检查数据源')

正常，未发现paper_type缺失现象


**(3) 检查 paper_type字段数据问题**

In [65]:
if 'paper_type' in data.columns:
    paper_type_number = 0
    
    paper_type = list(set(data['paper_type']))
    no_paper_type = data[data['paper_type'].isnull()]
    
    print('paper_type字段如下：\n')
    
    for eachtype in paper_type:
        eachtype_number = len(data[data['paper_type'] == eachtype])
        paper_type_number = paper_type_number + eachtype_number
        print(eachtype,'：',eachtype_number)
    print('\n论文总量：',len(data))
    print('拥有paper_type字段的论文数量：',paper_type_number)
    
    if paper_type_number != len(data):
        print('无paper_type字段的论文：')
        for i in range(len(no_paper_type)):
            print('url：',no_paper_type.iloc[i]['url'][0])
    else:
        print('正常，论文总数量和拥有paper_type字段的论文数量相等')
    
else:
    print('异常，可能缺少字段paper_type，请详细检查数据源')

paper_type字段如下：

report ： 17
letter ： 21
research-article ： 1834
correction ： 7
book-review ： 25
editorial ： 92
review-article ： 4
brief-report ： 22
opinion ： 10
obituary ： 1
introduction ： 1
abstract ： 4
other ： 424
case-report ： 17
article-commentary ： 5

论文总量： 2484
拥有paper_type字段的论文数量： 2484
正常，论文总数量和拥有paper_type字段的论文数量相等


## 6. 检查editor字段

In [66]:
if 'editor' in data.columns:
    no_editor = data[data['editor'].isnull()]
    if len(no_editor) == 0:
        print('正常，未发现editor字段缺失情况')
    else:
        print('异常，可能存在editor缺失现象')
        print('【editor数据类型可能缺失的数据量 / 数据总量】  <==>  【',len(no_editor),'/',len(data),'】')
        print('请从以下随机抽样检查，查看是否存在抓取遗漏：\n')
        for i in range(len(no_editor)):
            print('url：',no_editor.iloc[i]['url'][0])
else:
    print('异常，可能缺少字段editor，请详细检查数据源')

异常，可能缺少字段editor，请详细检查数据源


# =============
# 七. year、volume、issue、page_start、page_end、page_str字段查询
# =============

## 1. 检查year字段

**(1) 检查 year字段数据类型**

In [67]:
if 'year' in data.columns:
    if data['year'].dtypes == 'int64':
        print('正常，未发现year字段数据类型错误')
    else:
        print('异常，year字段数据类型可能错误，应该为int类型，数据集中是',data['year'].dtypes,'类型')
else:
    print('异常，可能缺少必填字段year')

正常，未发现year字段数据类型错误


**(2) 检查 year字段数据**

In [68]:
if 'year' in data.columns:
    year_list = sorted(list(set(data['year'])))
    no_year = data[data['year'].isnull()]
    year_paper_number = 0
    for year in year_list:
        year_data = data[data['year'] == year]
        year_paper_number = year_paper_number + len(year_data)
        if str(year) != 'nan':
            print('年份：' , year , '\t论文数：' , len(year_data))
    if len(data) == year_paper_number :
        print('\n正常，未发现year中数据量问题')
    else:
        print('\n异常，可能存在year缺失现象')
        print('【year可能缺失的数据量 / 数据总量】  <==>  【',year_paper_number,'/',len(data),'】')
        print('请详细检查以下数据：\n')
        for i in range(len(no_year)):
            print('url：',no_year.iloc[i]['url'][0])
else:
    print('异常，可能缺少必填字段year')

年份： 1978 	论文数： 29
年份： 1979 	论文数： 79
年份： 1980 	论文数： 60
年份： 1981 	论文数： 85
年份： 1982 	论文数： 64
年份： 1983 	论文数： 58
年份： 1984 	论文数： 57
年份： 1985 	论文数： 57
年份： 1986 	论文数： 56
年份： 1987 	论文数： 56
年份： 1988 	论文数： 52
年份： 1989 	论文数： 60
年份： 1990 	论文数： 48
年份： 1991 	论文数： 46
年份： 1992 	论文数： 48
年份： 1993 	论文数： 67
年份： 1994 	论文数： 53
年份： 1995 	论文数： 65
年份： 1996 	论文数： 51
年份： 1997 	论文数： 47
年份： 1998 	论文数： 53
年份： 1999 	论文数： 50
年份： 2000 	论文数： 48
年份： 2001 	论文数： 45
年份： 2002 	论文数： 23
年份： 2003 	论文数： 9
年份： 2004 	论文数： 22
年份： 2005 	论文数： 32
年份： 2006 	论文数： 44
年份： 2007 	论文数： 64
年份： 2008 	论文数： 103
年份： 2009 	论文数： 76
年份： 2010 	论文数： 73
年份： 2011 	论文数： 82
年份： 2012 	论文数： 81
年份： 2013 	论文数： 57
年份： 2014 	论文数： 71
年份： 2015 	论文数： 74
年份： 2016 	论文数： 78
年份： 2017 	论文数： 68
年份： 2018 	论文数： 99
年份： 2019 	论文数： 94

正常，未发现year中数据量问题


## 2. 检查volume、issue字段

**(1) 浏览检查 volume字段数据**

In [69]:
if 'volume' in data.columns:
    print(sorted(set(data['volume'])))
else:
    print('异常，可能缺少字段volume，请详细检查数据源')

[27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67]


**(2) 浏览检查 issue字段数据**

In [70]:
if 'issue' in data.columns:
    print(sorted(set(data['issue'])))
else:
    print('异常，可能缺少字段volume，请详细检查数据源')

[1, 2, 3, 4, 5, 6, 7, 8]


**(3) 检查 year、volume、issue字段数据**

In [71]:
if 'year' in data.columns:
    if 'volume' in data.columns:
        if 'issue' in data.columns:
            year_list = sorted(list(set(data['year'])))
            for year in year_list:
                year_data = data[data['year']==year]
                print('year：',year)
                if 'volume' not in list(data.columns): 
                    print('该年份无volume字段，请检查数据源' + '\n')
                else:
                    volume_list = sorted(list(set(year_data['volume'])))
                    for volume in volume_list:
                        volumedata = year_data[year_data['volume']==volume]
                        issue_list = sorted(list(set(volumedata['issue'])))
                        print('\t','volume：',volume,'\tissue：',issue_list)
        else:
             print('异常，可能缺少字段issue，请详细检查数据源')
    else:
        print('异常，可能缺少字段volume，请详细检查数据源')
else:
    print('异常，可能缺少必填字段year')

year： 1978
	 volume： 27 	issue： [2, 3]
year： 1979
	 volume： 27 	issue： [4, 6]
	 volume： 28 	issue： [1, 2, 3]
year： 1980
	 volume： 28 	issue： [6]
	 volume： 29 	issue： [1, 2, 3]
year： 1981
	 volume： 29 	issue： [4, 5, 6]
	 volume： 30 	issue： [1, 2, 3]
year： 1982
	 volume： 30 	issue： [4, 5, 6]
	 volume： 31 	issue： [1, 2, 3]
year： 1983
	 volume： 31 	issue： [4, 5, 6]
	 volume： 32 	issue： [1, 2, 3]
year： 1984
	 volume： 32 	issue： [4, 5, 6]
	 volume： 33 	issue： [1, 2, 3]
year： 1985
	 volume： 33 	issue： [4, 5, 6]
	 volume： 34 	issue： [1, 2, 3]
year： 1986
	 volume： 34 	issue： [4, 5, 6]
	 volume： 35 	issue： [1, 2, 3]
year： 1987
	 volume： 35 	issue： [4, 5, 6]
	 volume： 36 	issue： [1, 2, 3]
year： 1988
	 volume： 36 	issue： [4, 5, 6]
	 volume： 37 	issue： [1, 2, 3]
year： 1989
	 volume： 37 	issue： [4, 5, 6]
	 volume： 38 	issue： [1, 2, 3]
year： 1990
	 volume： 38 	issue： [4, 5, 6]
	 volume： 39 	issue： [1, 2, 3]
year： 1991
	 volume： 39 	issue： [4, 5, 6]
	 volume： 40 	issue： [1, 2, 3]
year： 1992
	 volume： 

## 3. 检查page字段

In [72]:
def pagecheck(pagetype):
    haspage = data[data[pagetype].notnull()]
    
    count = 0
    for eachindes in range(len(haspage)):
        if len(re.findall('[a-zA-Z]+', str(haspage[pagetype].iloc[eachindes]))) != 0:
            count+=1
            print(haspage[pagetype].iloc[eachindes])
            print(haspage['title'].iloc[eachindes])
            print(haspage['url'].iloc[eachindes][0])
            print('\n')
    if count == 0:
        print(pagetype,'中并没有发现字符！')

**(1) 检查 page_start字段数据缺失**

In [73]:
if 'page_start' in data.columns:
    no_page_start = data[data['page_start'].isnull()]
    if len(no_page_start) != 0:
        for i in range(len(no_page_start)):
            print('异常，可能出现page_start字段缺失情况，请详细检查数据源')
            print('url：',no_page_start.iloc[i]['url'][0],'\n')
    else:
        print('正常，未发现page_start字段缺失情况')
else:
    print('异常，可能缺少字段page_start，请详细检查数据源')

正常，未发现page_start字段缺失情况


**(2) 检查 page_start字段数据问题**

In [74]:
if 'page_start' not in list(data.columns): 
    print('所采集数据中无page_start字段，请检查数据源')
else:
    pagecheck('page_start')

ebi
Editorial Board Page
https://www.tandfonline.com/doi/abs/10.1080/01644300.1981.10393064


ebi
Editorial Board Page
https://www.tandfonline.com/doi/abs/10.1080/01644300.1980.10392950


ebi
Editorial Board EOV
https://www.tandfonline.com/doi/full/10.1080/07448481.2013.845728


ebi
Editorial Board Page
https://www.tandfonline.com/doi/abs/10.1080/01644300.1980.10392956


ebi
Editorial Board Page
https://www.tandfonline.com/doi/abs/10.1080/01644300.1979.10392854


ebi
Editorial Board Page
https://www.tandfonline.com/doi/abs/10.1080/01644300.1981.10393027


ebi
Editorial Board Page
https://www.tandfonline.com/doi/abs/10.1080/01644300.1981.10393011


ebi
Editorial Board Page
https://www.tandfonline.com/doi/abs/10.1080/07448481.1992.10392816


ebi
Editorial Board Page
https://www.tandfonline.com/doi/abs/10.1080/01644300.1979.10392869


ebi
Editorial Board Page
https://www.tandfonline.com/doi/abs/10.1080/07448481.1984.10392804


ebi
Editorial Board EOV
https://www.tandfonline.com/doi/full/1

**(3) 检查 page_end字段数据缺失**

In [70]:
if 'page_end' in data.columns:
    no_page_end = data[data['page_end'].isnull()]
    if len(no_page_end) != 0:
        for i in range(len(no_page_end)):
            print('异常，可能出现page_end字段缺失情况，请详细检查数据源')
            print('url：',no_page_end.iloc[i]['url'][0])
    else:
        print('正常，未发现page_end字段缺失情况')
else:
    print('异常，可能缺少字段page_end，请详细检查数据源')

正常，未发现page_end字段缺失情况


**(4) 检查 page_end字段数据问题**

In [75]:
if 'page_end' not in list(data.columns): 
    print('所采集数据中无page_end字段，请检查数据源')
else:
    pagecheck('page_end')

ebi
Editorial Board Page
https://www.tandfonline.com/doi/abs/10.1080/01644300.1981.10393064


ebi
Editorial Board Page
https://www.tandfonline.com/doi/abs/10.1080/01644300.1980.10392950


ebi
Editorial Board EOV
https://www.tandfonline.com/doi/full/10.1080/07448481.2013.845728


ebi
Editorial Board Page
https://www.tandfonline.com/doi/abs/10.1080/01644300.1980.10392956


ebi
Editorial Board Page
https://www.tandfonline.com/doi/abs/10.1080/01644300.1979.10392854


ebi
Editorial Board Page
https://www.tandfonline.com/doi/abs/10.1080/01644300.1981.10393027


ebi
Editorial Board Page
https://www.tandfonline.com/doi/abs/10.1080/01644300.1981.10393011


ebi
Editorial Board Page
https://www.tandfonline.com/doi/abs/10.1080/07448481.1992.10392816


ebi
Editorial Board Page
https://www.tandfonline.com/doi/abs/10.1080/01644300.1979.10392869


ebi
Editorial Board Page
https://www.tandfonline.com/doi/abs/10.1080/07448481.1984.10392804


ebi
Editorial Board EOV
https://www.tandfonline.com/doi/full/1

**(5) 检查 page字段数据量**

In [76]:
if 'page_start' and 'page_end' in data.columns:
    have_page_start = data[data['page_start'].notnull()]
    page_start_no_page_end = have_page_start[have_page_start['page_end'].isnull()]
    if len(page_start_no_page_end) != 0:
        print('异常，page_start和page_end字段数量可能不等，可能存在有page_start无page_end的情况')
        for i in range(len(page_start_no_page_end)):
            print('url：',page_start_no_page_end.iloc[i]['url'][0])
    else:
        print('正常，page_start字段数量 = page_end字段数量')
else:
    print('异常，可能缺少字段，请详细检查数据源')

正常，page_start字段数量 = page_end字段数量


**(6) 检查 page_str字段数据**

In [77]:
if 'page_str' in data.columns:
    print(set(data['page_str']))
else:
    print('异常，可能缺少字段page_str，请详细检查数据源')

{'83-85', '158-161', '292-293', '643-650', '116-124', '11-15', '583-596', '243-244', '518-524', '113', '293-298', '436-437', '281-289', '67', '181-184', '147-149', '103-109', '712', '799-808', '203-212', '55-67', '701-705', '277-282', '33-38', '691-698', '97-102', '461-470', '133-138', '85-92', '95-100', '93-95', '131-135', '120-123', '229-231', '57-58', '99-101', '281-284', '126-131', '195-202', '23-32', '315-316', '252-259', '608-616', '7-9', '604-612', '457-466', '132-135', '77-81', '513-522', '33-36', '167-174', '251', '403-414', '197-207', '391-392', '180-181', '281-290', '55-56', '448-458', '360-368', '242-244', '59-62', '102-108', '133-139', '23', '165-170', '126-127', '143', '38-40', '147-158', '7-11', '657-664', '185-189', '7-8', '227-230', '125', '193-197', '92-93', '34-36', '193', '317-323', '58-64', '180', '489-492', '58', '267-269', '492-501', '661-673', '262-265', '60-66', '90', '241-244', '611-614', '69-75', '813-816', '79-83', '213-216', '180-183', '140-141', '269-271',

## 4. 检查数据缺失情况

In [78]:
#将无page_str的数据先过滤掉
if 'page_str' not in list(data.columns): 
    print('所采集数据中无page_str字段，无法检查数据缺失情况，请检查数据源')
else:
    haspagestrdata = data[data['page_str'].notnull()]

    yearlist = sorted(list(set(data['year'])))
    for year in yearlist:
        print('year:',year)
        yeardata = haspagestrdata[haspagestrdata['year']==year]

        volumelist = sorted(list(set(yeardata['volume'])))
        for volume in volumelist:
            print('\t'+'volume:',volume)
            volumedata = yeardata[yeardata['volume']==volume]

            issuelist = sorted(list(set(volumedata['issue'])))
            #print(issuelist)
            for issue in issuelist:
                paperinissue = volumedata[volumedata['issue']==issue]
                num = len(paperinissue)
                print('\t'+'\t'+'issue:',issue,'paper number:',num)
                #pagestart = sorted(paperinissue['page_start'])[0]
                #print('\t','\t','\t','\t','page start:',pagestart)

                #------------------misssing---------------
                #1.将page_str切分成元组形式放入到列表中
                tuplelistpage = []
                zifutuplelistpage = []
                for eachpagestr in paperinissue['page_str']:
                    eachpagelist = str(eachpagestr).split('-')
                    if len(eachpagelist) == 2:
                        #print(eachpagelist)
                        if (eachpagelist[0].isdigit() and eachpagelist[1].isdigit()) == True:
                            tuplelistpage.append((int(eachpagelist[0]),int(eachpagelist[1])))
                        else:
                            zifutuplelistpage.append((eachpagelist[0],eachpagelist[1]))
                if len(zifutuplelistpage) != 0:
                    print('\t','\t','请检查该issu中是否存在如下额外page：',zifutuplelistpage)
                #2.对元组-列表中以第一位进行排序
                tuplelistpage = sorted(tuplelistpage)
                #print(tuplelistpage)
                if len(tuplelistpage) != 0:
                    pagestart = tuplelistpage[0][0]
                    print('\t','\t','\t','\t','page start:',pagestart)

                #3.对缺失的页面进行判断
                for i in range(len(tuplelistpage)-1):
                    #是否存在页码反复问题

                    #判断字符串中是否有字符
                    if (str(tuplelistpage[i][0]).isdigit() or str(tuplelistpage[i][1]).isdigit()) == True:
                        #1.同一论文中页码反复
                        if tuplelistpage[i][0]<=tuplelistpage[i][1]:
                            #2.临近论文中页码反复
                            if tuplelistpage[i][1] <= tuplelistpage[i+1][0]:
                                if tuplelistpage[i+1][0] - tuplelistpage[i][1] > 1:
    #                                 if tuplelistpage[i+1][0] - tuplelistpage[i][1] == 2:
    #                                     print('\t','\t','\t','\t','missing page :',tuplelistpage[i][1]+1)
    #                                 else:
    #                                     print('\t','\t','\t','\t','missing page range:',tuplelistpage[i][1]+1,'-',tuplelistpage[i+1][0]-1)
                                    if tuplelistpage[i+1][0] - tuplelistpage[i][1] > 2:
                                        print('\t','\t','\t','\t','missing page range:',tuplelistpage[i][1]+1,'-',tuplelistpage[i+1][0]-1)
                            else:
                                print('\t','\t','\t','\t','邻近paper存在页码反复，页码反复page：',tuplelistpage[i],tuplelistpage[i+1])
                                pgs1 = str(tuplelistpage[i][0])+'-'+str(tuplelistpage[i][1])
                                #print(pgs)
                                temp1 = paperinissue[paperinissue['page_str'] == pgs1]
                                print(np.array(temp1['title']),np.array(temp1['page_str']),np.array(temp1['url']))

                                pgs2 = str(tuplelistpage[i+1][0])+'-'+str(tuplelistpage[i+1][1])
                                temp2 = paperinissue[paperinissue['page_str'] == pgs2]
                                print(np.array(temp2['title']),np.array(temp2['page_str']),np.array(temp2['url']))
                        else:
                            print('\t','\t','\t','\t','同一paper中页码反复：',tuplelistpage[i])
                            pgs = str(tuplelistpage[i][0])+'-'+str(tuplelistpage[i][1])
                            print(paperinissue[paperinissue['page_str'] == pgs]['title'])
                #-----------------------------------------

                #pageend = sorted(paperinissue['page_end'])[-1]
                #print('\t','\t','\t','\t','page start:',pageend)
                if len(tuplelistpage) != 0:
                    pageend = tuplelistpage[-1][1]
                    print('\t','\t','\t','\t','page end:',pageend)

            print('\t'+'volume paper all number:',len(volumedata))

        print('The year paper all number:',len(yeardata))
        print('\n')

year: 1978
	volume: 27
		issue: 2 paper number: 16
	 	 	 	 page start: 72
	 	 	 	 邻近paper存在页码反复，页码反复page： (84, 97) (87, 110)
['The Definition and Treatment of Bulimarexia in College Women — A Pilot Study'] ['84-97'] [list(['https://www.tandfonline.com/doi/abs/10.1080/01644300.1978.10392831'])]
['Nonspecific Urethritis: Its Current Status'] ['87-110'] [list(['https://www.tandfonline.com/doi/abs/10.1080/01644300.1978.10392832'])]
	 	 	 	 邻近paper存在页码反复，页码反复page： (87, 110) (92, 97)
['Nonspecific Urethritis: Its Current Status'] ['87-110'] [list(['https://www.tandfonline.com/doi/abs/10.1080/01644300.1978.10392832'])]
['Mites and Wee Beasties'] ['92-97'] [list(['https://www.tandfonline.com/doi/abs/10.1080/01644300.1978.10392833'])]
	 	 	 	 邻近paper存在页码反复，页码反复page： (102, 106) (103, 104)
['Management of a Campus Rubella Epidemic'] ['102-106'] [list(['https://www.tandfonline.com/doi/abs/10.1080/01644300.1978.10392836'])]
['Management of Allergic Respiratory Disease by the Nurse Practitioner'] ['

	 	 	 	 missing page range: 224 - 226
	 	 	 	 page end: 240
		issue: 6 paper number: 11
	 	 	 	 page start: 247
	 	 	 	 missing page range: 270 - 271
	 	 	 	 missing page range: 276 - 277
	 	 	 	 missing page range: 290 - 291
	 	 	 	 page end: 301
	volume paper all number: 25
	volume: 41
		issue: 1 paper number: 7
	 	 	 	 page start: 3
	 	 	 	 page end: 36
		issue: 2 paper number: 9
	 	 	 	 page start: 43
	 	 	 	 missing page range: 63 - 66
	 	 	 	 missing page range: 73 - 74
	 	 	 	 page end: 84
		issue: 3 paper number: 7
	 	 	 	 page start: 91
	 	 	 	 missing page range: 111 - 114
	 	 	 	 page end: 131
	volume paper all number: 23
The year paper all number: 48


year: 1993
	volume: 41
		issue: 4 paper number: 10
	 	 	 	 page start: 139
	 	 	 	 missing page range: 167 - 168
	 	 	 	 page end: 180
		issue: 5 paper number: 9
	 	 	 	 page start: 187
	 	 	 	 missing page range: 221 - 222
	 	 	 	 page end: 229
		issue: 6 paper number: 22
	 	 	 	 page start: 235
	 	 	 	 missing page range: 2

	 	 	 	 page start: 105
	 	 	 	 page end: 120
	volume paper all number: 3
The year paper all number: 9


year: 2004
	volume: 52
		issue: 4 paper number: 2
	 	 	 	 page start: 159
	 	 	 	 missing page range: 169 - 180
	 	 	 	 page end: 188
		issue: 5 paper number: 5
	 	 	 	 page start: 197
	 	 	 	 page end: 236
		issue: 6 paper number: 3
	 	 	 	 page start: 245
	 	 	 	 page end: 267
	volume paper all number: 10
	volume: 53
		issue: 2 paper number: 8
	 	 	 	 page start: 53
	 	 	 	 missing page range: 69 - 70
	 	 	 	 missing page range: 92 - 94
	 	 	 	 page end: 96
		issue: 3 paper number: 4
	 	 	 	 page start: 101
	 	 	 	 page end: 132
	volume paper all number: 12
The year paper all number: 22


year: 2005
	volume: 53
		issue: 4 paper number: 6
	 	 	 	 page start: 149
	 	 	 	 missing page range: 189 - 190
	 	 	 	 page end: 192
		issue: 5 paper number: 3
	 	 	 	 page start: 219
	 	 	 	 page end: 238
		issue: 6 paper number: 5
	 	 	 	 page start: 245
	 	 	 	 page end: 290
	volume paper all